# 1. 더미화

# 1-1. Key값 , 더미화된 특성 먼저 제외

In [1]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
import tensorflow
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')
import sklearn.metrics as metrics
import warnings



from sklearn.datasets import load_boston
import xgboost
import matplotlib.pyplot as plt
from sklearn.metrics import explained_variance_score
from xgboost import plot_importance
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE



C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_selection.univariate_selection module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.ui

In [2]:
Child = pd.read_csv('./Child_model.csv',encoding = 'utf-8')
Child

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,개별사건번호,피해아동대상자,학대행위자대상,아동_성별,아동_생년월일,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,...,아조_분리조치,아조_원가정보호,아조_가정복귀,아조_사망,아조_기타_1,행조_고소/고발/사,행조_모니터링,행조_아동과의분리,행조_사건처리만나지,행조_만나지못함
0,0804080420141200220001,CASRCP20150110545,CASRCP20150120502,M,10~12세,내국인,NaN,NaN,기타,친부모,...,0,1,0,0,0,0,1,0,0,0
1,0806080620141200290001,CASRCP20150110903,CASRCP20150120827,F,13~15세,내국인,NaN,NaN,자택,친부모,...,0,1,0,0,0,1,0,0,0,0
2,0810010420141200070001,CASRCP20150210146,CASRCP20141221354,F,4~6세,내국인,NaN,NaN,자택,친부모,...,1,0,0,0,0,0,1,1,0,0
3,0104010420141200080001,CASRCP20150210367,CASRCP20141221398,F,16~17세,내국인,NaN,NaN,월세,친부,...,1,1,0,0,0,0,1,0,0,0
4,1103110320141200100001,CASRCP20140810269,CASRCP20150120430,F,1~3세,내국인,NaN,NaN,월세,친부,...,1,1,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87482,5506550620180165980001,CASRCP20180101087,CASRCP20180101089,M,10~12세,내국인,재학중,초등학교,NaN,친부모,...,0,1,0,0,0,0,1,0,0,0
87483,0805080520180100010001,CASRCP20180102176,CASRCP20180102182,M,16~17세,내국인,재학중,고등학교,자택,친부모,...,0,1,0,0,0,0,1,0,0,0
87484,5305530520180166200001,CASRCP20170111840,CASRCP20180103665,F,4~6세,내국인,NaN,NaN,보증금(전세)+월세,친부모,...,1,1,0,0,0,0,1,0,0,0
87485,5801580120180165450001,CASRCP20180100595,CASRCP20180100597,M,1~3세,내국인,NaN,어린이집,NaN,친모,...,0,1,0,0,0,0,1,0,0,0


In [3]:
for i in Child:
    print(i, Child[i].unique())

개별사건번호 ['0804080420141200220001' '0806080620141200290001'
 '0810010420141200070001' ... '5305530520180166200001'
 '5801580120180165450001' '5115511520180165080001']
피해아동대상자 ['CASRCP20150110545' 'CASRCP20150110903' 'CASRCP20150210146' ...
 'CASRCP20180102176' 'CASRCP20180100595' 'CASRCP20180100289']
학대행위자대상 ['CASRCP20150120502' 'CASRCP20150120827' 'CASRCP20141221354' ...
 'CASRCP20180102182' 'CASRCP20180103665' 'CASRCP20180100290']
아동_성별 ['M' 'F' nan]
아동_생년월일 ['10~12세' '13~15세' '4~6세' '16~17세' '1~3세' nan '7~9세' '0~1세미만' '18~20세']
아동_내국인여부 ['내국인' nan '외국인' '무국적']
아동_최종학력 [nan '재학중' '무학' '고등중퇴' '중등중퇴' '중등졸업' '초등졸업' '고등졸업' '초등중퇴']
아동_직업유형 [nan '초등학교' '중학교' '어린이집' '고등학교' '서비스 및 판매종사자' '무직' '기타' '유치원' '단순노무종사자'
 '전문가' '관리자' '기술공 및 준전문가' '사무종사자' '농림어업숙련종사자' '장치·기계조작 및 조립 종사자'
 '기능원 및 관련 기능 종사자']
아동_거주상태 ['기타' '자택' '월세' '보증금(전세)+월세' nan '무상' '전세' '영구임대아파트  또는 영구임대주택' '보호시설']
아동_친권자유형 ['친부모' '친부' '친모' nan '친권자 없음' '양부' '후견인 없음' '양부모' '양모']
아동_가족유형 ['친부모가정' '부자가족(이혼)' '동거(사실혼포함)' nan '부자가족(별거)' 

In [4]:
#key값 먼저 제외 
Chlid_key = Child[['개별사건번호', '피해아동대상자', '학대행위자대상','신대_통계거점','결과_조치결과일자','신고_접수일시']]

Child = Child.drop(['개별사건번호', '피해아동대상자', '학대행위자대상','신대_통계거점','결과_조치결과일자','신고_접수일시'],axis=1)

Child_result = Child[['아특_공격성', '아특_거짓말', '아특_장애의심',
       '아특_배우자폭력', '아특_장루·요루장애', '아특_기타장애', '아특_나태(무기력)', '아특_언어문제',
       '아특_뇌전증장애', '아특_청각장애', '아특_늦은귀가', '아특_어릴적학대경험', '아특_기타', '아특_우울',
       '아특_학습문제', '아특_언어장애', '아특_성문제', '아특_부적절한양육태', '아특_호흡기장애', '아특_양육지식및기술',
       '아특_뇌병변장애', '아특_잦은결석', '아특_틱장애', '아특_인터넷(게임)', '아특_알콜남용', '아특_폭력행동',
       '아특_영양결핍', '아특_자폐성장애', '아특_위생문제', '아특_정신질환및장애', '아특_부부및가족갈등',
       '아특_대인관계기피', '아특_존속학대', '아특_주의산만', '아특_약물', '아특_대소변문제', '아특_안면장애',
       '아특_도박_게임중독', '아특_탐식및결식', '아특_난(難)독해', '아특_흡연', '아특_사회적고립', '아특_정서문제',
       '아특_지체장애', '아특_무력감', '아특_반항', '아특_심장장애', '아특_경제적어려움', '아특_학교부적응',
       '아특_급만성질병', '아특_불건전한또래관', '아특_잦은병치례', '아특_지적장애', '아특_신체질환및장애',
       '아특_과잉행동', '아특_애착문제', '아특_정신장애', '아특_신체발달지연', '아특_스트레스', '아특_낮은자아존중감',
       '아특_오락중독', '아특_신장장애', '아특_허약', '아특_음주', '아특_도벽', '아특_시각장애', '아특_불안',
       '아특_간장애', '아특_무단결과', '아특_사회성문제', '아특_종교문제', '아특_원치않는아동', '아특_비행집단활동',
       '아특_난(難)작문', '아특_충동', '아특_가출', '아특_전과력', '아특_특성없음', '아특_성격및기질문제',
       '아조_분리조치', '아조_원가정보호', '아조_가정복귀', '아조_사망', '아조_기타_1', '행조_고소/고발/사',
       '행조_모니터링', '행조_아동과의분리', '행조_사건처리만나지', '행조_만나지못함']]

Child = Child.drop(['아특_공격성', '아특_거짓말', '아특_장애의심',
       '아특_배우자폭력', '아특_장루·요루장애', '아특_기타장애', '아특_나태(무기력)', '아특_언어문제',
       '아특_뇌전증장애', '아특_청각장애', '아특_늦은귀가', '아특_어릴적학대경험', '아특_기타', '아특_우울',
       '아특_학습문제', '아특_언어장애', '아특_성문제', '아특_부적절한양육태', '아특_호흡기장애', '아특_양육지식및기술',
       '아특_뇌병변장애', '아특_잦은결석', '아특_틱장애', '아특_인터넷(게임)', '아특_알콜남용', '아특_폭력행동',
       '아특_영양결핍', '아특_자폐성장애', '아특_위생문제', '아특_정신질환및장애', '아특_부부및가족갈등',
       '아특_대인관계기피', '아특_존속학대', '아특_주의산만', '아특_약물', '아특_대소변문제', '아특_안면장애',
       '아특_도박_게임중독', '아특_탐식및결식', '아특_난(難)독해', '아특_흡연', '아특_사회적고립', '아특_정서문제',
       '아특_지체장애', '아특_무력감', '아특_반항', '아특_심장장애', '아특_경제적어려움', '아특_학교부적응',
       '아특_급만성질병', '아특_불건전한또래관', '아특_잦은병치례', '아특_지적장애', '아특_신체질환및장애',
       '아특_과잉행동', '아특_애착문제', '아특_정신장애', '아특_신체발달지연', '아특_스트레스', '아특_낮은자아존중감',
       '아특_오락중독', '아특_신장장애', '아특_허약', '아특_음주', '아특_도벽', '아특_시각장애', '아특_불안',
       '아특_간장애', '아특_무단결과', '아특_사회성문제', '아특_종교문제', '아특_원치않는아동', '아특_비행집단활동',
       '아특_난(難)작문', '아특_충동', '아특_가출', '아특_전과력', '아특_특성없음', '아특_성격및기질문제',
       '아조_분리조치', '아조_원가정보호', '아조_가정복귀', '아조_사망', '아조_기타_1', '행조_고소/고발/사',
       '행조_모니터링', '행조_아동과의분리', '행조_사건처리만나지', '행조_만나지못함'],axis = 1)

In [5]:
Child.shape

(87487, 36)

In [6]:
pd.set_option("display.max_columns",None)
Child

,아동_성별,아동_생년월일,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,아동_가족유형,아동_다문화가족,아동_가구소득구분코,아동_기초생활수급유,아동_보호조치유형,신고_접수경로구분코,신고_신고자유형구분,신고_집단시설내사건,신고_접수유형,신고_신고접수구분,신고_피해아동상태구,신대_행위자아동관계,신대_아동동거여부,신대_접수유형,조사_집단시설내사건,조사_조사완료여부,조사_위험_점수,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,조대_특별관리유형,판단_유형_신체,판단_유형_정서,판단_유형_성,판단_유형_방임,판단_학대발생빈도,신고_접수연도,New_재학대혐의여부,New_Call_Count
0,M,10~12세,내국인,NaN,NaN,기타,친부모,친부모가정,일반,NaN,비수급권대상,NaN,112,비신고의무자,NaN,아동학대 의심사례,경찰접수,NaN,친부,동거,아동학대 의심사례,NaN,1,NaN,0,0,1,NaN,1,1,0,0,일회성,2014.0,N,1
1,F,13~15세,내국인,NaN,NaN,자택,친부모,친부모가정,일반,300만원이상,비수급권대상,NaN,112,비신고의무자,NaN,아동학대 의심사례,경찰접수,NaN,친부,동거,아동학대 의심사례,NaN,1,NaN,0,0,1,NaN,1,0,0,0,일회성,2014.0,Y,3
2,F,4~6세,내국인,NaN,NaN,자택,친부모,친부모가정,일반,150만원이상-200만원미만,수급권대상,NaN,아동보호전문기관일반전화,신고의무자,NaN,아동학대 의심사례,아동보호전문기관접수,NaN,친부,비동거,아동학대 의심사례,NaN,1,NaN,0,0,0,NaN,1,1,0,0,6개월에 한번,2014.0,N,1
3,F,16~17세,내국인,NaN,NaN,월세,친부,부자가족(이혼),일반,150만원이상-200만원미만,비수급권대상,NaN,112,비신고의무자,NaN,아동학대 의심사례,경찰접수,NaN,친부,동거,아동학대 의심사례,NaN,1,NaN,0,0,1,NaN,0,1,0,0,일회성,2014.0,N,1
4,F,1~3세,내국인,NaN,NaN,월세,친부,부자가족(이혼),일반,50만원미만,비수급권대상,NaN,아동보호전문기관일반전화,신고의무자,NaN,아동학대 의심사례,아동보호전문기관접수,NaN,부모의 동거인,동거,응급아동학대의심사례,NaN,1,NaN,1,0,1,NaN,1,0,0,1,거의 매일,2014.0,Y,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87482,M,10~12세,내국인,재학중,초등학교,NaN,친부모,친부모가정,일반,NaN,NaN,NaN,112,비신고의무자,해당사항없음,아동학대 의심사례,경찰접수,NaN,친부,동거,아동학대 의심사례,NaN,1,3.0,0,0,1,NaN,1,1,0,0,일회성,2018.0,N,1
87483,M,16~17세,내국인,재학중,고등학교,자택,친부모,친부모가정,일반,NaN,비수급권대상,NaN,아동보호전문기관일반전화,비신고의무자,해당사항없음,아동학대 의심사례,아동보호전문기관접수,NaN,친부,동거,아동학대 의심사례,NaN,1,3.0,0,0,1,NaN,0,1,0,0,일회성,2018.0,N,1
87484,F,4~6세,내국인,NaN,NaN,보증금(전세)+월세,친부모,친부모가정,일반,250만원이상-300만원미만,비수급권대상,NaN,112,비신고의무자,해당사항없음,아동학대 의심사례,경찰접수,NaN,친모,동거,아동학대 의심사례,NaN,1,4.0,1,0,1,NaN,0,0,0,1,6개월에 한번,2018.0,Y,3
87485,M,1~3세,내국인,NaN,어린이집,NaN,친모,모자가족(별거),일반,NaN,NaN,NaN,112,비신고의무자,해당사항없음,아동학대 의심사례,경찰접수,NaN,기타,동거,아동학대 의심사례,NaN,1,NaN,0,0,1,NaN,1,1,0,0,일회성,2018.0,N,1


# X생성

In [7]:
# 종속변수 제외 -> 독립변수들만 X로 저장
X = Child.drop(['New_재학대혐의여부'],axis = 1)
#y = Child['결과_학대혐의여부']

In [8]:
#X_call = X['신고_접수연도']
X = X.drop(['신고_접수연도'],axis=1)

In [9]:
X

,아동_성별,아동_생년월일,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,아동_가족유형,아동_다문화가족,아동_가구소득구분코,아동_기초생활수급유,아동_보호조치유형,신고_접수경로구분코,신고_신고자유형구분,신고_집단시설내사건,신고_접수유형,신고_신고접수구분,신고_피해아동상태구,신대_행위자아동관계,신대_아동동거여부,신대_접수유형,조사_집단시설내사건,조사_조사완료여부,조사_위험_점수,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,조대_특별관리유형,판단_유형_신체,판단_유형_정서,판단_유형_성,판단_유형_방임,판단_학대발생빈도,New_Call_Count
0,M,10~12세,내국인,NaN,NaN,기타,친부모,친부모가정,일반,NaN,비수급권대상,NaN,112,비신고의무자,NaN,아동학대 의심사례,경찰접수,NaN,친부,동거,아동학대 의심사례,NaN,1,NaN,0,0,1,NaN,1,1,0,0,일회성,1
1,F,13~15세,내국인,NaN,NaN,자택,친부모,친부모가정,일반,300만원이상,비수급권대상,NaN,112,비신고의무자,NaN,아동학대 의심사례,경찰접수,NaN,친부,동거,아동학대 의심사례,NaN,1,NaN,0,0,1,NaN,1,0,0,0,일회성,3
2,F,4~6세,내국인,NaN,NaN,자택,친부모,친부모가정,일반,150만원이상-200만원미만,수급권대상,NaN,아동보호전문기관일반전화,신고의무자,NaN,아동학대 의심사례,아동보호전문기관접수,NaN,친부,비동거,아동학대 의심사례,NaN,1,NaN,0,0,0,NaN,1,1,0,0,6개월에 한번,1
3,F,16~17세,내국인,NaN,NaN,월세,친부,부자가족(이혼),일반,150만원이상-200만원미만,비수급권대상,NaN,112,비신고의무자,NaN,아동학대 의심사례,경찰접수,NaN,친부,동거,아동학대 의심사례,NaN,1,NaN,0,0,1,NaN,0,1,0,0,일회성,1
4,F,1~3세,내국인,NaN,NaN,월세,친부,부자가족(이혼),일반,50만원미만,비수급권대상,NaN,아동보호전문기관일반전화,신고의무자,NaN,아동학대 의심사례,아동보호전문기관접수,NaN,부모의 동거인,동거,응급아동학대의심사례,NaN,1,NaN,1,0,1,NaN,1,0,0,1,거의 매일,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87482,M,10~12세,내국인,재학중,초등학교,NaN,친부모,친부모가정,일반,NaN,NaN,NaN,112,비신고의무자,해당사항없음,아동학대 의심사례,경찰접수,NaN,친부,동거,아동학대 의심사례,NaN,1,3.0,0,0,1,NaN,1,1,0,0,일회성,1
87483,M,16~17세,내국인,재학중,고등학교,자택,친부모,친부모가정,일반,NaN,비수급권대상,NaN,아동보호전문기관일반전화,비신고의무자,해당사항없음,아동학대 의심사례,아동보호전문기관접수,NaN,친부,동거,아동학대 의심사례,NaN,1,3.0,0,0,1,NaN,0,1,0,0,일회성,1
87484,F,4~6세,내국인,NaN,NaN,보증금(전세)+월세,친부모,친부모가정,일반,250만원이상-300만원미만,비수급권대상,NaN,112,비신고의무자,해당사항없음,아동학대 의심사례,경찰접수,NaN,친모,동거,아동학대 의심사례,NaN,1,4.0,1,0,1,NaN,0,0,0,1,6개월에 한번,3
87485,M,1~3세,내국인,NaN,어린이집,NaN,친모,모자가족(별거),일반,NaN,NaN,NaN,112,비신고의무자,해당사항없음,아동학대 의심사례,경찰접수,NaN,기타,동거,아동학대 의심사례,NaN,1,NaN,0,0,1,NaN,1,1,0,0,일회성,1


# 1-3. 더미화1 한것들 제외 :X1

In [10]:
#더미화 필요없는것들을 따로 빼놓고 더미화 이후 concat
X1 = X[['조사_조사완료여부','조대_재신고여부_1','조대_동일신고여부','조대_아동동거여부','판단_유형_신체','판단_유형_정서','판단_유형_성','판단_유형_방임']]
X = X.drop(['조사_조사완료여부','조대_재신고여부_1','조대_동일신고여부','조대_아동동거여부','판단_유형_신체','판단_유형_정서','판단_유형_성','판단_유형_방임'],axis=1)

# 1-4. get_dummies (더미화2)

In [11]:
#연속형도 더미화 하기위해 str로 변경
X = X.astype(str)

In [12]:
X = pd.get_dummies(X)

# 1-5. X = 더미화1 + 더미화2 + 더미화된 특성

In [13]:
#더미화한거랑, 더미화되어있던거랑(더미화작업필요없는변수들) 특성(이것도 사실상 더미화되어있는변수들) concat
X = pd.concat([X,X1,Child_result],axis = 1)
X

,아동_성별_F,아동_성별_M,아동_성별_nan,아동_생년월일_0~1세미만,아동_생년월일_10~12세,아동_생년월일_13~15세,아동_생년월일_16~17세,아동_생년월일_18~20세,아동_생년월일_1~3세,아동_생년월일_4~6세,아동_생년월일_7~9세,아동_생년월일_nan,아동_내국인여부_nan,아동_내국인여부_내국인,아동_내국인여부_무국적,아동_내국인여부_외국인,아동_최종학력_nan,아동_최종학력_고등졸업,아동_최종학력_고등중퇴,아동_최종학력_무학,아동_최종학력_재학중,아동_최종학력_중등졸업,아동_최종학력_중등중퇴,아동_최종학력_초등졸업,아동_최종학력_초등중퇴,아동_직업유형_nan,아동_직업유형_고등학교,아동_직업유형_관리자,아동_직업유형_기능원 및 관련 기능 종사자,아동_직업유형_기술공 및 준전문가,아동_직업유형_기타,아동_직업유형_농림어업숙련종사자,아동_직업유형_단순노무종사자,아동_직업유형_무직,아동_직업유형_사무종사자,아동_직업유형_서비스 및 판매종사자,아동_직업유형_어린이집,아동_직업유형_유치원,아동_직업유형_장치·기계조작 및 조립 종사자,아동_직업유형_전문가,아동_직업유형_중학교,아동_직업유형_초등학교,아동_거주상태_nan,아동_거주상태_기타,아동_거주상태_무상,아동_거주상태_보증금(전세)+월세,아동_거주상태_보호시설,아동_거주상태_영구임대아파트 또는 영구임대주택,아동_거주상태_월세,아동_거주상태_자택,아동_거주상태_전세,아동_친권자유형_nan,아동_친권자유형_양모,아동_친권자유형_양부,아동_친권자유형_양부모,아동_친권자유형_친권자 없음,아동_친권자유형_친모,아동_친권자유형_친부,아동_친권자유형_친부모,아동_친권자유형_후견인 없음,아동_가족유형_nan,아동_가족유형_기타,아동_가족유형_동거(사실혼포함),아동_가족유형_모자가족(가출),아동_가족유형_모자가족(별거),아동_가족유형_모자가족(사별),아동_가족유형_모자가족(이혼),아동_가족유형_미혼부·모가정,아동_가족유형_부자가족(가출),아동_가족유형_부자가족(별거),아동_가족유형_부자가족(사별),아동_가족유형_부자가족(이혼),아동_가족유형_소년소녀가정,아동_가족유형_시설보호,아동_가족유형_위탁가정,아동_가족유형_입양가정,아동_가족유형_재혼가정,아동_가족유형_친부모가정,아동_가족유형_친인척보호,아동_다문화가족_nan,아동_다문화가족_다문화,아동_다문화가족_북한이탈주민,아동_다문화가족_일반,아동_가구소득구분코_100만원이상-150만원미만,아동_가구소득구분코_150만원이상-200만원미만,아동_가구소득구분코_200만원이상-250만원미만,아동_가구소득구분코_250만원이상-300만원미만,아동_가구소득구분코_300만원이상,아동_가구소득구분코_50만원미만,아동_가구소득구분코_50만원이상~100만원미만,아동_가구소득구분코_nan,아동_기초생활수급유_nan,아동_기초생활수급유_비수급권대상,아동_기초생활수급유_수급권대상,아동_보호조치유형_1.0,아동_보호조치유형_nan,신고_접수경로구분코_112,신고_접수경로구분코_119,신고_접수경로구분코_129,신고_접수경로구분코_1366,신고_접수경로구분코_기타,신고_접수경로구분코_내방,신고_접수경로구분코_방문신고,신고_접수경로구분코_상담원 인지신고,신고_접수경로구분코_아동보호전문기관일반전화,신고_접수경로구분코_아동행복지원,신고_접수경로구분코_아동행복지원 상담원인지신고,신고_접수경로구분코_아동행복지원 읍면동인지신고,신고_접수경로구분코_인터넷,신고_접수경로구분코_일반전화,신고_접수경로구분코_타기관통보,신고_신고자유형구분_비신고의무자,신고_신고자유형구분_신고의무자,신고_집단시설내사건_nan,신고_집단시설내사건_교육기관,신고_집단시설내사건_기타,신고_집단시설내사건_시설,신고_집단시설내사건_어린이집,신고_집단시설내사건_해당사항없음,신고_접수유형_아동학대 의심사례,신고_접수유형_피해아동보호명령을 통한 인지,"신고_접수유형_학대행위자 법원명령을 통한 인지(임시조치, 보호처분)",신고_신고접수구분_경찰접수,신고_신고접수구분_아동보호전문기관접수,신고_피해아동상태구_nan,신고_피해아동상태구_아동사망,신고_피해아동상태구_중상해(의식불명 포함),신고_피해아동상태구_해당사항없음,신대_행위자아동관계_nan,신대_행위자아동관계_계모,신대_행위자아동관계_계부,신대_행위자아동관계_기타,신대_행위자아동관계_기타시설종사자,신대_행위자아동관계_낯선사람,신대_행위자아동관계_베이비시터(아이돌보미),신대_행위자아동관계_부모의 동거인,신대_행위자아동관계_아동복지시설종사자,신대_행위자아동관계_아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자,신대_행위자아동관계_양모,신대_행위자아동관계_양부,"신대_행위자아동관계_어린이집(보육교직원, 종사자)",신대_행위자아동관계_외조모,신대_행위자아동관계_외조부,신대_행위자아동관계_위탁모,신대_행위자아동관계_위탁부,"신대_행위자아동관계_유치원(교사,교직원, 종사자)",신대_행위자아동관계_이웃,신대_행위자아동관계_청소년관련시설종사자,신대_행위자아동관계_친모,신대_행위자아동관계_친부,신대_행위자아동관계_친인척,신대_행위자아동관계_친조모,신대_행위자아동관계_친조부,"신대_행위자아동관계_학교(교원, 교직원)","신대_행위자아동관계_학원(강사, 종사자)",신대_행위자아동관계_형제자매,신대_아동동거여부_nan,신대_아동동거여부_동거,신대_아동동거여부_비동거,신대_접수유형_동일신고,신대_접수유형_아동학대 의심사례,신대_접수유형_응급아동학대의심사례,조사_집단시설내사건_nan,조사_집단시설내사건_교육기관,조사_집단시설내사건_시설,조사_집단시설내사건_어린이집,조사_위험_점수_1.0,조사_위험_점수_2.0,조사_위험_점수_3.0,조사_위험_점수_4.0,조사_위험_점수_5.0,조사_위험_점수_6.0,조사_위험_점수_7.0,조사_위험_점수_8.0,조사_위험_점수_9.0,조사_위험_점수_nan,조대_특별관리유형_nan,조대_특별관리유형_가족재결합 평가도구 사용,조대_특별관리유형_공동모금회사업,조대_특별관리유형_아동인권지킴이단,조대_특별관리유형_종단연구,판단_학대발생빈도_1개월에 한번,판단_학대발생빈도_1년에 한번,판단_학대발생빈도_1주일에 한번,판단_학대발생빈도_2-3개월에 한번,판단_학대발생빈도_2~3일에 한번,판단_학대발생빈도_2년에 한번,판단_학대발생빈도_2주일에 한번,판단_학대발생빈도_3년에 한번,판단_학대발생빈도_6개월에 한번,판단_학대발생빈도_거의 매일,판단_학대발생빈도_일회성,New_Call_Count_1,New_Call_Count_10,New_Call_Count_11,New_Call_Count_12,New_Call_Count_13,New_Call_Count_14,New_Call_Count_15,New_Call_Count_16,New_Call_Count_17,New_Call_Count_2,New_Call_Count_3,New_Call_Count_4,New_Call_Count_5,New_Call_Count_6,New_Call_Count_7,New_Call_Count_8,New_Call_Count_9,조사_조사완료여부,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,판단_유형_신체,판단_유형_정서,판단_유형_성,판단_유형_방임,아특_공격성,아특_거짓말,아특_장애의심,아특_배우자폭력,아특_장루·요루장애,아특_기타장애,아특_나태(무기력),아특_언어문제,아특_뇌전증장애,아특_청각장애,아특_늦은귀가,아특_어릴적학대경험,아특_기타,아특_우울,아특_학습문제,아특_언어장애,아특_성문제,아특_부적절한양육태,아특_호흡기장애,아특_양육지식및기술,아특_뇌병변장애,아특_잦은결석,아특_틱장애,아특_인터넷(게임),아특_알콜남용,아특_폭력행동,아특_영양결핍,아특_자폐성장애,아특_위생문제,아특_정신질환및장애,아특_부부및가족갈등,아특_대인관계기피,아특_존속학대,아특_주의산만,아특_약물,아특_대소변문제,아특_안면장애,아특_도박_게임중독,아특_탐식및결식,아특_난(難)독해,아특_흡연,아특_사회적고립,아특_정서문제,아특_지체장애,아특_무력감,아특_반항,아특_심장장애,아특_경제적어려움,아특_학교부적응,아특_급만성질병,아특_불건전한또래관,아특_잦은병치례,아특_지적장애,아특_신체질환및장애,아특_과잉행동,아특_애착문제,아특_정신장애,아특_신체발달지연,아특_스트레스,아특_낮은자아존중감,아특_오락중독,아특_신장장애

In [14]:
for i in X:
    print(i, X[i].unique())

아동_성별_F [0 1]
아동_성별_M [1 0]
아동_성별_nan [0 1]
아동_생년월일_0~1세미만 [0 1]
아동_생년월일_10~12세 [1 0]
아동_생년월일_13~15세 [0 1]
아동_생년월일_16~17세 [0 1]
아동_생년월일_18~20세 [0 1]
아동_생년월일_1~3세 [0 1]
아동_생년월일_4~6세 [0 1]
아동_생년월일_7~9세 [0 1]
아동_생년월일_nan [0 1]
아동_내국인여부_nan [0 1]
아동_내국인여부_내국인 [1 0]
아동_내국인여부_무국적 [0 1]
아동_내국인여부_외국인 [0 1]
아동_최종학력_nan [1 0]
아동_최종학력_고등졸업 [0 1]
아동_최종학력_고등중퇴 [0 1]
아동_최종학력_무학 [0 1]
아동_최종학력_재학중 [0 1]
아동_최종학력_중등졸업 [0 1]
아동_최종학력_중등중퇴 [0 1]
아동_최종학력_초등졸업 [0 1]
아동_최종학력_초등중퇴 [0 1]
아동_직업유형_nan [1 0]
아동_직업유형_고등학교 [0 1]
아동_직업유형_관리자 [0 1]
아동_직업유형_기능원 및 관련 기능 종사자 [0 1]
아동_직업유형_기술공 및 준전문가 [0 1]
아동_직업유형_기타 [0 1]
아동_직업유형_농림어업숙련종사자 [0 1]
아동_직업유형_단순노무종사자 [0 1]
아동_직업유형_무직 [0 1]
아동_직업유형_사무종사자 [0 1]
아동_직업유형_서비스 및 판매종사자 [0 1]
아동_직업유형_어린이집 [0 1]
아동_직업유형_유치원 [0 1]
아동_직업유형_장치·기계조작 및 조립 종사자 [0 1]
아동_직업유형_전문가 [0 1]
아동_직업유형_중학교 [0 1]
아동_직업유형_초등학교 [0 1]
아동_거주상태_nan [0 1]
아동_거주상태_기타 [1 0]
아동_거주상태_무상 [0 1]
아동_거주상태_보증금(전세)+월세 [0 1]
아동_거주상태_보호시설 [0 1]
아동_거주상태_영구임대아파트  또는 영구임대주택 [0 1]
아동_거주상태_월세 [0 1]
아동_거주상태_자택 [0 1]
아동_거주상태_전세 

In [15]:
for i in X.columns:
    print(i)

아동_성별_F
아동_성별_M
아동_성별_nan
아동_생년월일_0~1세미만
아동_생년월일_10~12세
아동_생년월일_13~15세
아동_생년월일_16~17세
아동_생년월일_18~20세
아동_생년월일_1~3세
아동_생년월일_4~6세
아동_생년월일_7~9세
아동_생년월일_nan
아동_내국인여부_nan
아동_내국인여부_내국인
아동_내국인여부_무국적
아동_내국인여부_외국인
아동_최종학력_nan
아동_최종학력_고등졸업
아동_최종학력_고등중퇴
아동_최종학력_무학
아동_최종학력_재학중
아동_최종학력_중등졸업
아동_최종학력_중등중퇴
아동_최종학력_초등졸업
아동_최종학력_초등중퇴
아동_직업유형_nan
아동_직업유형_고등학교
아동_직업유형_관리자
아동_직업유형_기능원 및 관련 기능 종사자
아동_직업유형_기술공 및 준전문가
아동_직업유형_기타
아동_직업유형_농림어업숙련종사자
아동_직업유형_단순노무종사자
아동_직업유형_무직
아동_직업유형_사무종사자
아동_직업유형_서비스 및 판매종사자
아동_직업유형_어린이집
아동_직업유형_유치원
아동_직업유형_장치·기계조작 및 조립 종사자
아동_직업유형_전문가
아동_직업유형_중학교
아동_직업유형_초등학교
아동_거주상태_nan
아동_거주상태_기타
아동_거주상태_무상
아동_거주상태_보증금(전세)+월세
아동_거주상태_보호시설
아동_거주상태_영구임대아파트  또는 영구임대주택
아동_거주상태_월세
아동_거주상태_자택
아동_거주상태_전세
아동_친권자유형_nan
아동_친권자유형_양모
아동_친권자유형_양부
아동_친권자유형_양부모
아동_친권자유형_친권자 없음
아동_친권자유형_친모
아동_친권자유형_친부
아동_친권자유형_친부모
아동_친권자유형_후견인 없음
아동_가족유형_nan
아동_가족유형_기타
아동_가족유형_동거(사실혼포함)
아동_가족유형_모자가족(가출)
아동_가족유형_모자가족(별거)
아동_가족유형_모자가족(사별)
아동_가족유형_모자가족(이혼)
아동_가족유형_미혼부·모가정
아동_가족유형_부자가족(가출)
아동_가족유형_부자가족(별거)
아동_가족유형_부자가족(사별)
아동_가족

# 1-6. nan 컬럼 제외

In [16]:
for i in X.columns:
    if (i[-1] == 'n'):
        X = X.drop([i],axis = 1)

In [17]:
#분석을 위해 다시 float으로 type변경
X = X.astype(float)

# 2. 모델 저장

In [18]:
#저장하기
import os
os.getcwd()

'C:\\Look After'

In [19]:
#불러오기
import joblib
log_clf = joblib.load('logistic_model.pkl') 
xgb_model = joblib.load('xgb_model.pkl') 
forest = joblib.load('forest_model.pkl') 

In [20]:
def model_avg(data):
    
    # Xgboost
    test1 = np.array(data) # np.array형식으로 바꾼다.
    nam = col_name
    
    # test를 데이터 프레임 형태로 변환
    test1 =pd.DataFrame(test1.reshape(len(test1) // len(nam), len(nam)), columns=nam)

    # 입력값의 예측값 출력
    

    prob1 = round(xgb_model.predict(test1)[0]*100,2)
    print("Xgboost모델에 입력한 아동의 학대 발생률은 {}%입니다.".format(prob1))
    

    # Logistic regression
    test2 = np.array([data])
    
    log_clf.predict_proba(test2)[0][1]
    prob2 = round(log_clf.predict_proba(test2)[0][1]*100,2)
    print("Losigtic regression모델에 입력한 아동의 학대 발생률은 {}%입니다.".format(prob2))
    
    
    # Random forest
    test3 = np.array([data])
    
    forest.predict_proba(test3)[0][1]
    prob3 = round(forest.predict_proba(test3)[0][1]*100,2)
    print("Random forest모델에 입력한 아동의 학대 발생률은 {}%입니다.".format(prob3))
    
    print("\n입력한 아동의 학대 발생률의 평균은 {}%입니다.\n\n".format(round((prob1+prob2+prob3)/3.0,2)))

    return round((prob1+prob2+prob3)/3.0,2)

# 3. 재학대 예측확률 함수

# 3-1. 통계거점 붙이기

In [21]:
#더미화한거랑, 더미화되어있던거랑(더미화작업필요없는변수들) 특성(이것도 사실상 더미화되어있는변수들) concat
X = pd.concat([Chlid_key['신대_통계거점'],X],axis = 1)
X

,신대_통계거점,아동_성별_F,아동_성별_M,아동_생년월일_0~1세미만,아동_생년월일_10~12세,아동_생년월일_13~15세,아동_생년월일_16~17세,아동_생년월일_18~20세,아동_생년월일_1~3세,아동_생년월일_4~6세,아동_생년월일_7~9세,아동_내국인여부_내국인,아동_내국인여부_무국적,아동_내국인여부_외국인,아동_최종학력_고등졸업,아동_최종학력_고등중퇴,아동_최종학력_무학,아동_최종학력_재학중,아동_최종학력_중등졸업,아동_최종학력_중등중퇴,아동_최종학력_초등졸업,아동_최종학력_초등중퇴,아동_직업유형_고등학교,아동_직업유형_관리자,아동_직업유형_기능원 및 관련 기능 종사자,아동_직업유형_기술공 및 준전문가,아동_직업유형_기타,아동_직업유형_농림어업숙련종사자,아동_직업유형_단순노무종사자,아동_직업유형_무직,아동_직업유형_사무종사자,아동_직업유형_서비스 및 판매종사자,아동_직업유형_어린이집,아동_직업유형_유치원,아동_직업유형_장치·기계조작 및 조립 종사자,아동_직업유형_전문가,아동_직업유형_중학교,아동_직업유형_초등학교,아동_거주상태_기타,아동_거주상태_무상,아동_거주상태_보증금(전세)+월세,아동_거주상태_보호시설,아동_거주상태_영구임대아파트 또는 영구임대주택,아동_거주상태_월세,아동_거주상태_자택,아동_거주상태_전세,아동_친권자유형_양모,아동_친권자유형_양부,아동_친권자유형_양부모,아동_친권자유형_친권자 없음,아동_친권자유형_친모,아동_친권자유형_친부,아동_친권자유형_친부모,아동_친권자유형_후견인 없음,아동_가족유형_기타,아동_가족유형_동거(사실혼포함),아동_가족유형_모자가족(가출),아동_가족유형_모자가족(별거),아동_가족유형_모자가족(사별),아동_가족유형_모자가족(이혼),아동_가족유형_미혼부·모가정,아동_가족유형_부자가족(가출),아동_가족유형_부자가족(별거),아동_가족유형_부자가족(사별),아동_가족유형_부자가족(이혼),아동_가족유형_소년소녀가정,아동_가족유형_시설보호,아동_가족유형_위탁가정,아동_가족유형_입양가정,아동_가족유형_재혼가정,아동_가족유형_친부모가정,아동_가족유형_친인척보호,아동_다문화가족_다문화,아동_다문화가족_북한이탈주민,아동_다문화가족_일반,아동_가구소득구분코_100만원이상-150만원미만,아동_가구소득구분코_150만원이상-200만원미만,아동_가구소득구분코_200만원이상-250만원미만,아동_가구소득구분코_250만원이상-300만원미만,아동_가구소득구분코_300만원이상,아동_가구소득구분코_50만원미만,아동_가구소득구분코_50만원이상~100만원미만,아동_기초생활수급유_비수급권대상,아동_기초생활수급유_수급권대상,아동_보호조치유형_1.0,신고_접수경로구분코_112,신고_접수경로구분코_119,신고_접수경로구분코_129,신고_접수경로구분코_1366,신고_접수경로구분코_기타,신고_접수경로구분코_내방,신고_접수경로구분코_방문신고,신고_접수경로구분코_상담원 인지신고,신고_접수경로구분코_아동보호전문기관일반전화,신고_접수경로구분코_아동행복지원,신고_접수경로구분코_아동행복지원 상담원인지신고,신고_접수경로구분코_아동행복지원 읍면동인지신고,신고_접수경로구분코_인터넷,신고_접수경로구분코_일반전화,신고_접수경로구분코_타기관통보,신고_신고자유형구분_비신고의무자,신고_신고자유형구분_신고의무자,신고_집단시설내사건_교육기관,신고_집단시설내사건_기타,신고_집단시설내사건_시설,신고_집단시설내사건_어린이집,신고_집단시설내사건_해당사항없음,신고_접수유형_아동학대 의심사례,신고_접수유형_피해아동보호명령을 통한 인지,"신고_접수유형_학대행위자 법원명령을 통한 인지(임시조치, 보호처분)",신고_신고접수구분_경찰접수,신고_신고접수구분_아동보호전문기관접수,신고_피해아동상태구_아동사망,신고_피해아동상태구_중상해(의식불명 포함),신고_피해아동상태구_해당사항없음,신대_행위자아동관계_계모,신대_행위자아동관계_계부,신대_행위자아동관계_기타,신대_행위자아동관계_기타시설종사자,신대_행위자아동관계_낯선사람,신대_행위자아동관계_베이비시터(아이돌보미),신대_행위자아동관계_부모의 동거인,신대_행위자아동관계_아동복지시설종사자,신대_행위자아동관계_아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자,신대_행위자아동관계_양모,신대_행위자아동관계_양부,"신대_행위자아동관계_어린이집(보육교직원, 종사자)",신대_행위자아동관계_외조모,신대_행위자아동관계_외조부,신대_행위자아동관계_위탁모,신대_행위자아동관계_위탁부,"신대_행위자아동관계_유치원(교사,교직원, 종사자)",신대_행위자아동관계_이웃,신대_행위자아동관계_청소년관련시설종사자,신대_행위자아동관계_친모,신대_행위자아동관계_친부,신대_행위자아동관계_친인척,신대_행위자아동관계_친조모,신대_행위자아동관계_친조부,"신대_행위자아동관계_학교(교원, 교직원)","신대_행위자아동관계_학원(강사, 종사자)",신대_행위자아동관계_형제자매,신대_아동동거여부_동거,신대_아동동거여부_비동거,신대_접수유형_동일신고,신대_접수유형_아동학대 의심사례,신대_접수유형_응급아동학대의심사례,조사_집단시설내사건_교육기관,조사_집단시설내사건_시설,조사_집단시설내사건_어린이집,조사_위험_점수_1.0,조사_위험_점수_2.0,조사_위험_점수_3.0,조사_위험_점수_4.0,조사_위험_점수_5.0,조사_위험_점수_6.0,조사_위험_점수_7.0,조사_위험_점수_8.0,조사_위험_점수_9.0,조대_특별관리유형_가족재결합 평가도구 사용,조대_특별관리유형_공동모금회사업,조대_특별관리유형_아동인권지킴이단,조대_특별관리유형_종단연구,판단_학대발생빈도_1개월에 한번,판단_학대발생빈도_1년에 한번,판단_학대발생빈도_1주일에 한번,판단_학대발생빈도_2-3개월에 한번,판단_학대발생빈도_2~3일에 한번,판단_학대발생빈도_2년에 한번,판단_학대발생빈도_2주일에 한번,판단_학대발생빈도_3년에 한번,판단_학대발생빈도_6개월에 한번,판단_학대발생빈도_거의 매일,판단_학대발생빈도_일회성,New_Call_Count_1,New_Call_Count_10,New_Call_Count_11,New_Call_Count_12,New_Call_Count_13,New_Call_Count_14,New_Call_Count_15,New_Call_Count_16,New_Call_Count_17,New_Call_Count_2,New_Call_Count_3,New_Call_Count_4,New_Call_Count_5,New_Call_Count_6,New_Call_Count_7,New_Call_Count_8,New_Call_Count_9,조사_조사완료여부,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,판단_유형_신체,판단_유형_정서,판단_유형_성,판단_유형_방임,아특_공격성,아특_거짓말,아특_장애의심,아특_배우자폭력,아특_장루·요루장애,아특_기타장애,아특_나태(무기력),아특_언어문제,아특_뇌전증장애,아특_청각장애,아특_늦은귀가,아특_어릴적학대경험,아특_기타,아특_우울,아특_학습문제,아특_언어장애,아특_성문제,아특_부적절한양육태,아특_호흡기장애,아특_양육지식및기술,아특_뇌병변장애,아특_잦은결석,아특_틱장애,아특_인터넷(게임),아특_알콜남용,아특_폭력행동,아특_영양결핍,아특_자폐성장애,아특_위생문제,아특_정신질환및장애,아특_부부및가족갈등,아특_대인관계기피,아특_존속학대,아특_주의산만,아특_약물,아특_대소변문제,아특_안면장애,아특_도박_게임중독,아특_탐식및결식,아특_난(難)독해,아특_흡연,아특_사회적고립,아특_정서문제,아특_지체장애,아특_무력감,아특_반항,아특_심장장애,아특_경제적어려움,아특_학교부적응,아특_급만성질병,아특_불건전한또래관,아특_잦은병치례,아특_지적장애,아특_신체질환및장애,아특_과잉행동,아특_애착문제,아특_정신장애,아특_신체발달지연,아특_스트레스,아특_낮은자아존중감,아특_오락중독,아특_신장장애,아특_허약,아특_음주,아특_도벽,아특_시각장애,아특_불안,아특_간장애,아특_무단결과,아특_사회성문제,아특_종교문제,아특_원치않는아동,아특_비행집단활동,아특_난(難)작문,아특_충동,아특_가출,아특_전과력,아특_특성없음,아특_성격및기질문제,아조_분리조치,아조_원가정보호,아조_가정복귀,아조_사망,아조_기타_1,행조_고소/고발/사,행조_모니터링,행조_아동과의분리,행조_사건처리만나지,행조_만나지못함
0,경기,0.0,1.0,0.0,1.0,0.0,

In [22]:
col_name = X.drop(['신대_통계거점'],axis=1).columns
X

,신대_통계거점,아동_성별_F,아동_성별_M,아동_생년월일_0~1세미만,아동_생년월일_10~12세,아동_생년월일_13~15세,아동_생년월일_16~17세,아동_생년월일_18~20세,아동_생년월일_1~3세,아동_생년월일_4~6세,아동_생년월일_7~9세,아동_내국인여부_내국인,아동_내국인여부_무국적,아동_내국인여부_외국인,아동_최종학력_고등졸업,아동_최종학력_고등중퇴,아동_최종학력_무학,아동_최종학력_재학중,아동_최종학력_중등졸업,아동_최종학력_중등중퇴,아동_최종학력_초등졸업,아동_최종학력_초등중퇴,아동_직업유형_고등학교,아동_직업유형_관리자,아동_직업유형_기능원 및 관련 기능 종사자,아동_직업유형_기술공 및 준전문가,아동_직업유형_기타,아동_직업유형_농림어업숙련종사자,아동_직업유형_단순노무종사자,아동_직업유형_무직,아동_직업유형_사무종사자,아동_직업유형_서비스 및 판매종사자,아동_직업유형_어린이집,아동_직업유형_유치원,아동_직업유형_장치·기계조작 및 조립 종사자,아동_직업유형_전문가,아동_직업유형_중학교,아동_직업유형_초등학교,아동_거주상태_기타,아동_거주상태_무상,아동_거주상태_보증금(전세)+월세,아동_거주상태_보호시설,아동_거주상태_영구임대아파트 또는 영구임대주택,아동_거주상태_월세,아동_거주상태_자택,아동_거주상태_전세,아동_친권자유형_양모,아동_친권자유형_양부,아동_친권자유형_양부모,아동_친권자유형_친권자 없음,아동_친권자유형_친모,아동_친권자유형_친부,아동_친권자유형_친부모,아동_친권자유형_후견인 없음,아동_가족유형_기타,아동_가족유형_동거(사실혼포함),아동_가족유형_모자가족(가출),아동_가족유형_모자가족(별거),아동_가족유형_모자가족(사별),아동_가족유형_모자가족(이혼),아동_가족유형_미혼부·모가정,아동_가족유형_부자가족(가출),아동_가족유형_부자가족(별거),아동_가족유형_부자가족(사별),아동_가족유형_부자가족(이혼),아동_가족유형_소년소녀가정,아동_가족유형_시설보호,아동_가족유형_위탁가정,아동_가족유형_입양가정,아동_가족유형_재혼가정,아동_가족유형_친부모가정,아동_가족유형_친인척보호,아동_다문화가족_다문화,아동_다문화가족_북한이탈주민,아동_다문화가족_일반,아동_가구소득구분코_100만원이상-150만원미만,아동_가구소득구분코_150만원이상-200만원미만,아동_가구소득구분코_200만원이상-250만원미만,아동_가구소득구분코_250만원이상-300만원미만,아동_가구소득구분코_300만원이상,아동_가구소득구분코_50만원미만,아동_가구소득구분코_50만원이상~100만원미만,아동_기초생활수급유_비수급권대상,아동_기초생활수급유_수급권대상,아동_보호조치유형_1.0,신고_접수경로구분코_112,신고_접수경로구분코_119,신고_접수경로구분코_129,신고_접수경로구분코_1366,신고_접수경로구분코_기타,신고_접수경로구분코_내방,신고_접수경로구분코_방문신고,신고_접수경로구분코_상담원 인지신고,신고_접수경로구분코_아동보호전문기관일반전화,신고_접수경로구분코_아동행복지원,신고_접수경로구분코_아동행복지원 상담원인지신고,신고_접수경로구분코_아동행복지원 읍면동인지신고,신고_접수경로구분코_인터넷,신고_접수경로구분코_일반전화,신고_접수경로구분코_타기관통보,신고_신고자유형구분_비신고의무자,신고_신고자유형구분_신고의무자,신고_집단시설내사건_교육기관,신고_집단시설내사건_기타,신고_집단시설내사건_시설,신고_집단시설내사건_어린이집,신고_집단시설내사건_해당사항없음,신고_접수유형_아동학대 의심사례,신고_접수유형_피해아동보호명령을 통한 인지,"신고_접수유형_학대행위자 법원명령을 통한 인지(임시조치, 보호처분)",신고_신고접수구분_경찰접수,신고_신고접수구분_아동보호전문기관접수,신고_피해아동상태구_아동사망,신고_피해아동상태구_중상해(의식불명 포함),신고_피해아동상태구_해당사항없음,신대_행위자아동관계_계모,신대_행위자아동관계_계부,신대_행위자아동관계_기타,신대_행위자아동관계_기타시설종사자,신대_행위자아동관계_낯선사람,신대_행위자아동관계_베이비시터(아이돌보미),신대_행위자아동관계_부모의 동거인,신대_행위자아동관계_아동복지시설종사자,신대_행위자아동관계_아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자,신대_행위자아동관계_양모,신대_행위자아동관계_양부,"신대_행위자아동관계_어린이집(보육교직원, 종사자)",신대_행위자아동관계_외조모,신대_행위자아동관계_외조부,신대_행위자아동관계_위탁모,신대_행위자아동관계_위탁부,"신대_행위자아동관계_유치원(교사,교직원, 종사자)",신대_행위자아동관계_이웃,신대_행위자아동관계_청소년관련시설종사자,신대_행위자아동관계_친모,신대_행위자아동관계_친부,신대_행위자아동관계_친인척,신대_행위자아동관계_친조모,신대_행위자아동관계_친조부,"신대_행위자아동관계_학교(교원, 교직원)","신대_행위자아동관계_학원(강사, 종사자)",신대_행위자아동관계_형제자매,신대_아동동거여부_동거,신대_아동동거여부_비동거,신대_접수유형_동일신고,신대_접수유형_아동학대 의심사례,신대_접수유형_응급아동학대의심사례,조사_집단시설내사건_교육기관,조사_집단시설내사건_시설,조사_집단시설내사건_어린이집,조사_위험_점수_1.0,조사_위험_점수_2.0,조사_위험_점수_3.0,조사_위험_점수_4.0,조사_위험_점수_5.0,조사_위험_점수_6.0,조사_위험_점수_7.0,조사_위험_점수_8.0,조사_위험_점수_9.0,조대_특별관리유형_가족재결합 평가도구 사용,조대_특별관리유형_공동모금회사업,조대_특별관리유형_아동인권지킴이단,조대_특별관리유형_종단연구,판단_학대발생빈도_1개월에 한번,판단_학대발생빈도_1년에 한번,판단_학대발생빈도_1주일에 한번,판단_학대발생빈도_2-3개월에 한번,판단_학대발생빈도_2~3일에 한번,판단_학대발생빈도_2년에 한번,판단_학대발생빈도_2주일에 한번,판단_학대발생빈도_3년에 한번,판단_학대발생빈도_6개월에 한번,판단_학대발생빈도_거의 매일,판단_학대발생빈도_일회성,New_Call_Count_1,New_Call_Count_10,New_Call_Count_11,New_Call_Count_12,New_Call_Count_13,New_Call_Count_14,New_Call_Count_15,New_Call_Count_16,New_Call_Count_17,New_Call_Count_2,New_Call_Count_3,New_Call_Count_4,New_Call_Count_5,New_Call_Count_6,New_Call_Count_7,New_Call_Count_8,New_Call_Count_9,조사_조사완료여부,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,판단_유형_신체,판단_유형_정서,판단_유형_성,판단_유형_방임,아특_공격성,아특_거짓말,아특_장애의심,아특_배우자폭력,아특_장루·요루장애,아특_기타장애,아특_나태(무기력),아특_언어문제,아특_뇌전증장애,아특_청각장애,아특_늦은귀가,아특_어릴적학대경험,아특_기타,아특_우울,아특_학습문제,아특_언어장애,아특_성문제,아특_부적절한양육태,아특_호흡기장애,아특_양육지식및기술,아특_뇌병변장애,아특_잦은결석,아특_틱장애,아특_인터넷(게임),아특_알콜남용,아특_폭력행동,아특_영양결핍,아특_자폐성장애,아특_위생문제,아특_정신질환및장애,아특_부부및가족갈등,아특_대인관계기피,아특_존속학대,아특_주의산만,아특_약물,아특_대소변문제,아특_안면장애,아특_도박_게임중독,아특_탐식및결식,아특_난(難)독해,아특_흡연,아특_사회적고립,아특_정서문제,아특_지체장애,아특_무력감,아특_반항,아특_심장장애,아특_경제적어려움,아특_학교부적응,아특_급만성질병,아특_불건전한또래관,아특_잦은병치례,아특_지적장애,아특_신체질환및장애,아특_과잉행동,아특_애착문제,아특_정신장애,아특_신체발달지연,아특_스트레스,아특_낮은자아존중감,아특_오락중독,아특_신장장애,아특_허약,아특_음주,아특_도벽,아특_시각장애,아특_불안,아특_간장애,아특_무단결과,아특_사회성문제,아특_종교문제,아특_원치않는아동,아특_비행집단활동,아특_난(難)작문,아특_충동,아특_가출,아특_전과력,아특_특성없음,아특_성격및기질문제,아조_분리조치,아조_원가정보호,아조_가정복귀,아조_사망,아조_기타_1,행조_고소/고발/사,행조_모니터링,행조_아동과의분리,행조_사건처리만나지,행조_만나지못함
0,경기,0.0,1.0,0.0,1.0,0.0,

# 한 아동의 재학대 확률 예측

In [23]:
#예시쓰는방법
import numpy as np
test = []
for i in col_name:
    test.append(0)

In [24]:
model_avg(test)

Xgboost모델에 입력한 아동의 학대 발생률은 95.0%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 71.67%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 54.03%입니다.

입력한 아동의 학대 발생률의 평균은 73.57%입니다.




73.57

# 경남지역 아동의 재학대 확률 예측

X

In [25]:
test = X[X['신대_통계거점']=='서울']
test

,신대_통계거점,아동_성별_F,아동_성별_M,아동_생년월일_0~1세미만,아동_생년월일_10~12세,아동_생년월일_13~15세,아동_생년월일_16~17세,아동_생년월일_18~20세,아동_생년월일_1~3세,아동_생년월일_4~6세,아동_생년월일_7~9세,아동_내국인여부_내국인,아동_내국인여부_무국적,아동_내국인여부_외국인,아동_최종학력_고등졸업,아동_최종학력_고등중퇴,아동_최종학력_무학,아동_최종학력_재학중,아동_최종학력_중등졸업,아동_최종학력_중등중퇴,아동_최종학력_초등졸업,아동_최종학력_초등중퇴,아동_직업유형_고등학교,아동_직업유형_관리자,아동_직업유형_기능원 및 관련 기능 종사자,아동_직업유형_기술공 및 준전문가,아동_직업유형_기타,아동_직업유형_농림어업숙련종사자,아동_직업유형_단순노무종사자,아동_직업유형_무직,아동_직업유형_사무종사자,아동_직업유형_서비스 및 판매종사자,아동_직업유형_어린이집,아동_직업유형_유치원,아동_직업유형_장치·기계조작 및 조립 종사자,아동_직업유형_전문가,아동_직업유형_중학교,아동_직업유형_초등학교,아동_거주상태_기타,아동_거주상태_무상,아동_거주상태_보증금(전세)+월세,아동_거주상태_보호시설,아동_거주상태_영구임대아파트 또는 영구임대주택,아동_거주상태_월세,아동_거주상태_자택,아동_거주상태_전세,아동_친권자유형_양모,아동_친권자유형_양부,아동_친권자유형_양부모,아동_친권자유형_친권자 없음,아동_친권자유형_친모,아동_친권자유형_친부,아동_친권자유형_친부모,아동_친권자유형_후견인 없음,아동_가족유형_기타,아동_가족유형_동거(사실혼포함),아동_가족유형_모자가족(가출),아동_가족유형_모자가족(별거),아동_가족유형_모자가족(사별),아동_가족유형_모자가족(이혼),아동_가족유형_미혼부·모가정,아동_가족유형_부자가족(가출),아동_가족유형_부자가족(별거),아동_가족유형_부자가족(사별),아동_가족유형_부자가족(이혼),아동_가족유형_소년소녀가정,아동_가족유형_시설보호,아동_가족유형_위탁가정,아동_가족유형_입양가정,아동_가족유형_재혼가정,아동_가족유형_친부모가정,아동_가족유형_친인척보호,아동_다문화가족_다문화,아동_다문화가족_북한이탈주민,아동_다문화가족_일반,아동_가구소득구분코_100만원이상-150만원미만,아동_가구소득구분코_150만원이상-200만원미만,아동_가구소득구분코_200만원이상-250만원미만,아동_가구소득구분코_250만원이상-300만원미만,아동_가구소득구분코_300만원이상,아동_가구소득구분코_50만원미만,아동_가구소득구분코_50만원이상~100만원미만,아동_기초생활수급유_비수급권대상,아동_기초생활수급유_수급권대상,아동_보호조치유형_1.0,신고_접수경로구분코_112,신고_접수경로구분코_119,신고_접수경로구분코_129,신고_접수경로구분코_1366,신고_접수경로구분코_기타,신고_접수경로구분코_내방,신고_접수경로구분코_방문신고,신고_접수경로구분코_상담원 인지신고,신고_접수경로구분코_아동보호전문기관일반전화,신고_접수경로구분코_아동행복지원,신고_접수경로구분코_아동행복지원 상담원인지신고,신고_접수경로구분코_아동행복지원 읍면동인지신고,신고_접수경로구분코_인터넷,신고_접수경로구분코_일반전화,신고_접수경로구분코_타기관통보,신고_신고자유형구분_비신고의무자,신고_신고자유형구분_신고의무자,신고_집단시설내사건_교육기관,신고_집단시설내사건_기타,신고_집단시설내사건_시설,신고_집단시설내사건_어린이집,신고_집단시설내사건_해당사항없음,신고_접수유형_아동학대 의심사례,신고_접수유형_피해아동보호명령을 통한 인지,"신고_접수유형_학대행위자 법원명령을 통한 인지(임시조치, 보호처분)",신고_신고접수구분_경찰접수,신고_신고접수구분_아동보호전문기관접수,신고_피해아동상태구_아동사망,신고_피해아동상태구_중상해(의식불명 포함),신고_피해아동상태구_해당사항없음,신대_행위자아동관계_계모,신대_행위자아동관계_계부,신대_행위자아동관계_기타,신대_행위자아동관계_기타시설종사자,신대_행위자아동관계_낯선사람,신대_행위자아동관계_베이비시터(아이돌보미),신대_행위자아동관계_부모의 동거인,신대_행위자아동관계_아동복지시설종사자,신대_행위자아동관계_아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자,신대_행위자아동관계_양모,신대_행위자아동관계_양부,"신대_행위자아동관계_어린이집(보육교직원, 종사자)",신대_행위자아동관계_외조모,신대_행위자아동관계_외조부,신대_행위자아동관계_위탁모,신대_행위자아동관계_위탁부,"신대_행위자아동관계_유치원(교사,교직원, 종사자)",신대_행위자아동관계_이웃,신대_행위자아동관계_청소년관련시설종사자,신대_행위자아동관계_친모,신대_행위자아동관계_친부,신대_행위자아동관계_친인척,신대_행위자아동관계_친조모,신대_행위자아동관계_친조부,"신대_행위자아동관계_학교(교원, 교직원)","신대_행위자아동관계_학원(강사, 종사자)",신대_행위자아동관계_형제자매,신대_아동동거여부_동거,신대_아동동거여부_비동거,신대_접수유형_동일신고,신대_접수유형_아동학대 의심사례,신대_접수유형_응급아동학대의심사례,조사_집단시설내사건_교육기관,조사_집단시설내사건_시설,조사_집단시설내사건_어린이집,조사_위험_점수_1.0,조사_위험_점수_2.0,조사_위험_점수_3.0,조사_위험_점수_4.0,조사_위험_점수_5.0,조사_위험_점수_6.0,조사_위험_점수_7.0,조사_위험_점수_8.0,조사_위험_점수_9.0,조대_특별관리유형_가족재결합 평가도구 사용,조대_특별관리유형_공동모금회사업,조대_특별관리유형_아동인권지킴이단,조대_특별관리유형_종단연구,판단_학대발생빈도_1개월에 한번,판단_학대발생빈도_1년에 한번,판단_학대발생빈도_1주일에 한번,판단_학대발생빈도_2-3개월에 한번,판단_학대발생빈도_2~3일에 한번,판단_학대발생빈도_2년에 한번,판단_학대발생빈도_2주일에 한번,판단_학대발생빈도_3년에 한번,판단_학대발생빈도_6개월에 한번,판단_학대발생빈도_거의 매일,판단_학대발생빈도_일회성,New_Call_Count_1,New_Call_Count_10,New_Call_Count_11,New_Call_Count_12,New_Call_Count_13,New_Call_Count_14,New_Call_Count_15,New_Call_Count_16,New_Call_Count_17,New_Call_Count_2,New_Call_Count_3,New_Call_Count_4,New_Call_Count_5,New_Call_Count_6,New_Call_Count_7,New_Call_Count_8,New_Call_Count_9,조사_조사완료여부,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,판단_유형_신체,판단_유형_정서,판단_유형_성,판단_유형_방임,아특_공격성,아특_거짓말,아특_장애의심,아특_배우자폭력,아특_장루·요루장애,아특_기타장애,아특_나태(무기력),아특_언어문제,아특_뇌전증장애,아특_청각장애,아특_늦은귀가,아특_어릴적학대경험,아특_기타,아특_우울,아특_학습문제,아특_언어장애,아특_성문제,아특_부적절한양육태,아특_호흡기장애,아특_양육지식및기술,아특_뇌병변장애,아특_잦은결석,아특_틱장애,아특_인터넷(게임),아특_알콜남용,아특_폭력행동,아특_영양결핍,아특_자폐성장애,아특_위생문제,아특_정신질환및장애,아특_부부및가족갈등,아특_대인관계기피,아특_존속학대,아특_주의산만,아특_약물,아특_대소변문제,아특_안면장애,아특_도박_게임중독,아특_탐식및결식,아특_난(難)독해,아특_흡연,아특_사회적고립,아특_정서문제,아특_지체장애,아특_무력감,아특_반항,아특_심장장애,아특_경제적어려움,아특_학교부적응,아특_급만성질병,아특_불건전한또래관,아특_잦은병치례,아특_지적장애,아특_신체질환및장애,아특_과잉행동,아특_애착문제,아특_정신장애,아특_신체발달지연,아특_스트레스,아특_낮은자아존중감,아특_오락중독,아특_신장장애,아특_허약,아특_음주,아특_도벽,아특_시각장애,아특_불안,아특_간장애,아특_무단결과,아특_사회성문제,아특_종교문제,아특_원치않는아동,아특_비행집단활동,아특_난(難)작문,아특_충동,아특_가출,아특_전과력,아특_특성없음,아특_성격및기질문제,아조_분리조치,아조_원가정보호,아조_가정복귀,아조_사망,아조_기타_1,행조_고소/고발/사,행조_모니터링,행조_아동과의분리,행조_사건처리만나지,행조_만나지못함
2,서울,1.0,0.0,0.0,0.0,0.0,

In [26]:
prob = test.drop('신대_통계거점',axis=1)
prob

,아동_성별_F,아동_성별_M,아동_생년월일_0~1세미만,아동_생년월일_10~12세,아동_생년월일_13~15세,아동_생년월일_16~17세,아동_생년월일_18~20세,아동_생년월일_1~3세,아동_생년월일_4~6세,아동_생년월일_7~9세,아동_내국인여부_내국인,아동_내국인여부_무국적,아동_내국인여부_외국인,아동_최종학력_고등졸업,아동_최종학력_고등중퇴,아동_최종학력_무학,아동_최종학력_재학중,아동_최종학력_중등졸업,아동_최종학력_중등중퇴,아동_최종학력_초등졸업,아동_최종학력_초등중퇴,아동_직업유형_고등학교,아동_직업유형_관리자,아동_직업유형_기능원 및 관련 기능 종사자,아동_직업유형_기술공 및 준전문가,아동_직업유형_기타,아동_직업유형_농림어업숙련종사자,아동_직업유형_단순노무종사자,아동_직업유형_무직,아동_직업유형_사무종사자,아동_직업유형_서비스 및 판매종사자,아동_직업유형_어린이집,아동_직업유형_유치원,아동_직업유형_장치·기계조작 및 조립 종사자,아동_직업유형_전문가,아동_직업유형_중학교,아동_직업유형_초등학교,아동_거주상태_기타,아동_거주상태_무상,아동_거주상태_보증금(전세)+월세,아동_거주상태_보호시설,아동_거주상태_영구임대아파트 또는 영구임대주택,아동_거주상태_월세,아동_거주상태_자택,아동_거주상태_전세,아동_친권자유형_양모,아동_친권자유형_양부,아동_친권자유형_양부모,아동_친권자유형_친권자 없음,아동_친권자유형_친모,아동_친권자유형_친부,아동_친권자유형_친부모,아동_친권자유형_후견인 없음,아동_가족유형_기타,아동_가족유형_동거(사실혼포함),아동_가족유형_모자가족(가출),아동_가족유형_모자가족(별거),아동_가족유형_모자가족(사별),아동_가족유형_모자가족(이혼),아동_가족유형_미혼부·모가정,아동_가족유형_부자가족(가출),아동_가족유형_부자가족(별거),아동_가족유형_부자가족(사별),아동_가족유형_부자가족(이혼),아동_가족유형_소년소녀가정,아동_가족유형_시설보호,아동_가족유형_위탁가정,아동_가족유형_입양가정,아동_가족유형_재혼가정,아동_가족유형_친부모가정,아동_가족유형_친인척보호,아동_다문화가족_다문화,아동_다문화가족_북한이탈주민,아동_다문화가족_일반,아동_가구소득구분코_100만원이상-150만원미만,아동_가구소득구분코_150만원이상-200만원미만,아동_가구소득구분코_200만원이상-250만원미만,아동_가구소득구분코_250만원이상-300만원미만,아동_가구소득구분코_300만원이상,아동_가구소득구분코_50만원미만,아동_가구소득구분코_50만원이상~100만원미만,아동_기초생활수급유_비수급권대상,아동_기초생활수급유_수급권대상,아동_보호조치유형_1.0,신고_접수경로구분코_112,신고_접수경로구분코_119,신고_접수경로구분코_129,신고_접수경로구분코_1366,신고_접수경로구분코_기타,신고_접수경로구분코_내방,신고_접수경로구분코_방문신고,신고_접수경로구분코_상담원 인지신고,신고_접수경로구분코_아동보호전문기관일반전화,신고_접수경로구분코_아동행복지원,신고_접수경로구분코_아동행복지원 상담원인지신고,신고_접수경로구분코_아동행복지원 읍면동인지신고,신고_접수경로구분코_인터넷,신고_접수경로구분코_일반전화,신고_접수경로구분코_타기관통보,신고_신고자유형구분_비신고의무자,신고_신고자유형구분_신고의무자,신고_집단시설내사건_교육기관,신고_집단시설내사건_기타,신고_집단시설내사건_시설,신고_집단시설내사건_어린이집,신고_집단시설내사건_해당사항없음,신고_접수유형_아동학대 의심사례,신고_접수유형_피해아동보호명령을 통한 인지,"신고_접수유형_학대행위자 법원명령을 통한 인지(임시조치, 보호처분)",신고_신고접수구분_경찰접수,신고_신고접수구분_아동보호전문기관접수,신고_피해아동상태구_아동사망,신고_피해아동상태구_중상해(의식불명 포함),신고_피해아동상태구_해당사항없음,신대_행위자아동관계_계모,신대_행위자아동관계_계부,신대_행위자아동관계_기타,신대_행위자아동관계_기타시설종사자,신대_행위자아동관계_낯선사람,신대_행위자아동관계_베이비시터(아이돌보미),신대_행위자아동관계_부모의 동거인,신대_행위자아동관계_아동복지시설종사자,신대_행위자아동관계_아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자,신대_행위자아동관계_양모,신대_행위자아동관계_양부,"신대_행위자아동관계_어린이집(보육교직원, 종사자)",신대_행위자아동관계_외조모,신대_행위자아동관계_외조부,신대_행위자아동관계_위탁모,신대_행위자아동관계_위탁부,"신대_행위자아동관계_유치원(교사,교직원, 종사자)",신대_행위자아동관계_이웃,신대_행위자아동관계_청소년관련시설종사자,신대_행위자아동관계_친모,신대_행위자아동관계_친부,신대_행위자아동관계_친인척,신대_행위자아동관계_친조모,신대_행위자아동관계_친조부,"신대_행위자아동관계_학교(교원, 교직원)","신대_행위자아동관계_학원(강사, 종사자)",신대_행위자아동관계_형제자매,신대_아동동거여부_동거,신대_아동동거여부_비동거,신대_접수유형_동일신고,신대_접수유형_아동학대 의심사례,신대_접수유형_응급아동학대의심사례,조사_집단시설내사건_교육기관,조사_집단시설내사건_시설,조사_집단시설내사건_어린이집,조사_위험_점수_1.0,조사_위험_점수_2.0,조사_위험_점수_3.0,조사_위험_점수_4.0,조사_위험_점수_5.0,조사_위험_점수_6.0,조사_위험_점수_7.0,조사_위험_점수_8.0,조사_위험_점수_9.0,조대_특별관리유형_가족재결합 평가도구 사용,조대_특별관리유형_공동모금회사업,조대_특별관리유형_아동인권지킴이단,조대_특별관리유형_종단연구,판단_학대발생빈도_1개월에 한번,판단_학대발생빈도_1년에 한번,판단_학대발생빈도_1주일에 한번,판단_학대발생빈도_2-3개월에 한번,판단_학대발생빈도_2~3일에 한번,판단_학대발생빈도_2년에 한번,판단_학대발생빈도_2주일에 한번,판단_학대발생빈도_3년에 한번,판단_학대발생빈도_6개월에 한번,판단_학대발생빈도_거의 매일,판단_학대발생빈도_일회성,New_Call_Count_1,New_Call_Count_10,New_Call_Count_11,New_Call_Count_12,New_Call_Count_13,New_Call_Count_14,New_Call_Count_15,New_Call_Count_16,New_Call_Count_17,New_Call_Count_2,New_Call_Count_3,New_Call_Count_4,New_Call_Count_5,New_Call_Count_6,New_Call_Count_7,New_Call_Count_8,New_Call_Count_9,조사_조사완료여부,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,판단_유형_신체,판단_유형_정서,판단_유형_성,판단_유형_방임,아특_공격성,아특_거짓말,아특_장애의심,아특_배우자폭력,아특_장루·요루장애,아특_기타장애,아특_나태(무기력),아특_언어문제,아특_뇌전증장애,아특_청각장애,아특_늦은귀가,아특_어릴적학대경험,아특_기타,아특_우울,아특_학습문제,아특_언어장애,아특_성문제,아특_부적절한양육태,아특_호흡기장애,아특_양육지식및기술,아특_뇌병변장애,아특_잦은결석,아특_틱장애,아특_인터넷(게임),아특_알콜남용,아특_폭력행동,아특_영양결핍,아특_자폐성장애,아특_위생문제,아특_정신질환및장애,아특_부부및가족갈등,아특_대인관계기피,아특_존속학대,아특_주의산만,아특_약물,아특_대소변문제,아특_안면장애,아특_도박_게임중독,아특_탐식및결식,아특_난(難)독해,아특_흡연,아특_사회적고립,아특_정서문제,아특_지체장애,아특_무력감,아특_반항,아특_심장장애,아특_경제적어려움,아특_학교부적응,아특_급만성질병,아특_불건전한또래관,아특_잦은병치례,아특_지적장애,아특_신체질환및장애,아특_과잉행동,아특_애착문제,아특_정신장애,아특_신체발달지연,아특_스트레스,아특_낮은자아존중감,아특_오락중독,아특_신장장애,아특_허약,아특_음주,아특_도벽,아특_시각장애,아특_불안,아특_간장애,아특_무단결과,아특_사회성문제,아특_종교문제,아특_원치않는아동,아특_비행집단활동,아특_난(難)작문,아특_충동,아특_가출,아특_전과력,아특_특성없음,아특_성격및기질문제,아조_분리조치,아조_원가정보호,아조_가정복귀,아조_사망,아조_기타_1,행조_고소/고발/사,행조_모니터링,행조_아동과의분리,행조_사건처리만나지,행조_만나지못함
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [27]:
for i in prob:
    print(i,prob[i].unique())

아동_성별_F [1. 0.]
아동_성별_M [0. 1.]
아동_생년월일_0~1세미만 [0. 1.]
아동_생년월일_10~12세 [0. 1.]
아동_생년월일_13~15세 [0. 1.]
아동_생년월일_16~17세 [0. 1.]
아동_생년월일_18~20세 [0. 1.]
아동_생년월일_1~3세 [0. 1.]
아동_생년월일_4~6세 [1. 0.]
아동_생년월일_7~9세 [0. 1.]
아동_내국인여부_내국인 [1. 0.]
아동_내국인여부_무국적 [0.]
아동_내국인여부_외국인 [0. 1.]
아동_최종학력_고등졸업 [0. 1.]
아동_최종학력_고등중퇴 [0. 1.]
아동_최종학력_무학 [0. 1.]
아동_최종학력_재학중 [0. 1.]
아동_최종학력_중등졸업 [0. 1.]
아동_최종학력_중등중퇴 [0. 1.]
아동_최종학력_초등졸업 [0. 1.]
아동_최종학력_초등중퇴 [0. 1.]
아동_직업유형_고등학교 [0. 1.]
아동_직업유형_관리자 [0.]
아동_직업유형_기능원 및 관련 기능 종사자 [0.]
아동_직업유형_기술공 및 준전문가 [0.]
아동_직업유형_기타 [0. 1.]
아동_직업유형_농림어업숙련종사자 [0.]
아동_직업유형_단순노무종사자 [0. 1.]
아동_직업유형_무직 [0. 1.]
아동_직업유형_사무종사자 [0. 1.]
아동_직업유형_서비스 및 판매종사자 [0. 1.]
아동_직업유형_어린이집 [0. 1.]
아동_직업유형_유치원 [0. 1.]
아동_직업유형_장치·기계조작 및 조립 종사자 [0.]
아동_직업유형_전문가 [0.]
아동_직업유형_중학교 [0. 1.]
아동_직업유형_초등학교 [0. 1.]
아동_거주상태_기타 [0. 1.]
아동_거주상태_무상 [0. 1.]
아동_거주상태_보증금(전세)+월세 [0. 1.]
아동_거주상태_보호시설 [0. 1.]
아동_거주상태_영구임대아파트  또는 영구임대주택 [0. 1.]
아동_거주상태_월세 [0. 1.]
아동_거주상태_자택 [1. 0.]
아동_거주상태_전세 [0. 1.]
아동_친권자유형_양모 [0. 1.]
아동_친권자유형_양부 

In [ ]:
init_col = pd.DataFrame

In [28]:
local_prob=[]
for i in range(len(prob)):
    local_prob.append(model_avg(prob.iloc[i,:]))

Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.03%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 29.33%입니다.

입력한 아동의 학대 발생률의 평균은 10.58%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.02%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 26.31%입니다.

입력한 아동의 학대 발생률의 평균은 9.57%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 79.81%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 79.66%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 66.89%입니다.

입력한 아동의 학대 발생률의 평균은 75.45%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 89.14%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 93.18%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 68.91%입니다.

입력한 아동의 학대 발생률의 평균은 83.74%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.03%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 23.95%입니다.

입력한 아동의 학대 발생률의 평균은 8.79%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.02%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 25.63%입니다.

입력한 아동의 학대 발생률의 평균은 9.34%입니다.


Xgboost모델에 입력한 

Xgboost모델에 입력한 아동의 학대 발생률은 88.64%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 93.79%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 71.71%입니다.

입력한 아동의 학대 발생률의 평균은 84.71%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.08%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 31.03%입니다.

입력한 아동의 학대 발생률의 평균은 11.16%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.01%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 26.52%입니다.

입력한 아동의 학대 발생률의 평균은 9.64%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.05%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 34.57%입니다.

입력한 아동의 학대 발생률의 평균은 12.33%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.04%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 29.25%입니다.

입력한 아동의 학대 발생률의 평균은 10.56%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 93.31%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 94.94%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 68.27%입니다.

입력한 아동의 학대 발생률의 평균은 85.51%입니다.


Xgboost모델에 입력

Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.11%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 31.44%입니다.

입력한 아동의 학대 발생률의 평균은 11.31%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.02%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 42.19%입니다.

입력한 아동의 학대 발생률의 평균은 14.86%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 93.47%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 92.49%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 72.52%입니다.

입력한 아동의 학대 발생률의 평균은 86.16%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.02%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 25.16%입니다.

입력한 아동의 학대 발생률의 평균은 9.19%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.01%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 24.25%입니다.

입력한 아동의 학대 발생률의 평균은 8.88%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 93.99%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 97.89%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 70.84%입니다.

입력한 아동의 학대 발생률의 평균은 87.57%입니다.


Xgboost모델에 입력한


Xgboost모델에 입력한 아동의 학대 발생률은 89.77%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 94.38%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 71.57%입니다.

입력한 아동의 학대 발생률의 평균은 85.24%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 89.69%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 92.53%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 71.84%입니다.

입력한 아동의 학대 발생률의 평균은 84.69%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 89.43%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 92.38%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 74.49%입니다.

입력한 아동의 학대 발생률의 평균은 85.43%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 93.74%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 97.32%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 64.61%입니다.

입력한 아동의 학대 발생률의 평균은 85.22%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 89.09%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 93.22%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 68.03%입니다.

입력한 아동의 학대 발생률의 평균은 83.45%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 89.09%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 92.71%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 67.53%입니다.

입력한 아동의 학대 발생률의 평균은 83.11%입니다.


Xgb

Random forest모델에 입력한 아동의 학대 발생률은 25.56%입니다.

입력한 아동의 학대 발생률의 평균은 9.32%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.08%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 29.98%입니다.

입력한 아동의 학대 발생률의 평균은 10.81%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.01%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 24.91%입니다.

입력한 아동의 학대 발생률의 평균은 9.1%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 91.4%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 92.94%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 76.6%입니다.

입력한 아동의 학대 발생률의 평균은 86.98%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.05%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 29.91%입니다.

입력한 아동의 학대 발생률의 평균은 10.78%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 94.22%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 94.87%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 68.97%입니다.

입력한 아동의 학대 발생률의 평균은 86.02%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.05%입니다.
Random forest모델에 

...

Random forest모델에 입력한 아동의 학대 발생률은 71.69%입니다.

입력한 아동의 학대 발생률의 평균은 83.09%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.03%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 29.07%입니다.

입력한 아동의 학대 발생률의 평균은 10.49%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.04%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 30.61%입니다.

입력한 아동의 학대 발생률의 평균은 11.01%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.04%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 30.28%입니다.

입력한 아동의 학대 발생률의 평균은 10.9%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 96.45%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 98.9%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 68.2%입니다.

입력한 아동의 학대 발생률의 평균은 87.85%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 92.09%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 95.48%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 69.39%입니다.

입력한 아동의 학대 발생률의 평균은 85.65%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 93.58%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 96.41%입니다.
Random forest

Random forest모델에 입력한 아동의 학대 발생률은 21.14%입니다.

입력한 아동의 학대 발생률의 평균은 7.85%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 88.92%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 85.84%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 62.36%입니다.

입력한 아동의 학대 발생률의 평균은 79.04%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.06%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 35.97%입니다.

입력한 아동의 학대 발생률의 평균은 12.8%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 91.56%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 92.21%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 59.83%입니다.

입력한 아동의 학대 발생률의 평균은 81.2%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.02%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 28.23%입니다.

입력한 아동의 학대 발생률의 평균은 10.21%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.02%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 21.48%입니다.

입력한 아동의 학대 발생률의 평균은 7.96%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.04%입니다.
Random forest모델에

Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.03%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 30.66%입니다.

입력한 아동의 학대 발생률의 평균은 11.02%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.04%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 25.03%입니다.

입력한 아동의 학대 발생률의 평균은 9.15%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.14%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 31.67%입니다.

입력한 아동의 학대 발생률의 평균은 11.4%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.07%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 31.36%입니다.

입력한 아동의 학대 발생률의 평균은 11.27%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.04%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 22.87%입니다.

입력한 아동의 학대 발생률의 평균은 8.43%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.02%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 34.26%입니다.

입력한 아동의 학대 발생률의 평균은 12.22%입니다.


Xgboost모델에 입력한 아동의 

Random forest모델에 입력한 아동의 학대 발생률은 29.71%입니다.

입력한 아동의 학대 발생률의 평균은 10.71%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.06%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 30.36%입니다.

입력한 아동의 학대 발생률의 평균은 10.93%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 92.86%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 96.23%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 65.06%입니다.

입력한 아동의 학대 발생률의 평균은 84.72%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.04%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 23.65%입니다.

입력한 아동의 학대 발생률의 평균은 8.69%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.05%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 23.57%입니다.

입력한 아동의 학대 발생률의 평균은 8.67%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.05%입니다.
Random forest모델에 입력한 아동의 학대 발생률은 25.92%입니다.

입력한 아동의 학대 발생률의 평균은 9.45%입니다.


Xgboost모델에 입력한 아동의 학대 발생률은 2.38%입니다.
Losigtic regression모델에 입력한 아동의 학대 발생률은 0.04%입니다.
Random forest모델에 

KeyboardInterrupt: 

In [29]:
columns_lst = []
for i in X_train.columns:
    columns_lst.append(i)

columns_lst

NameError: name 'X_train' is not defined

In [ ]:
['아동_성별_F',
 '아동_성별_M',
 '아동_생년월일_0~1세미만',
 '아동_생년월일_10~12세',
 '아동_생년월일_13~15세',
 '아동_생년월일_16~17세',
 '아동_생년월일_18~20세',
 '아동_생년월일_1~3세',
 '아동_생년월일_4~6세',
 '아동_생년월일_7~9세',
 '아동_내국인여부_내국인',
 '아동_내국인여부_무국적',
 '아동_내국인여부_외국인',
 '아동_최종학력_고등졸업',
 '아동_최종학력_고등중퇴',
 '아동_최종학력_무학',
 '아동_최종학력_재학중',
 '아동_최종학력_중등졸업',
 '아동_최종학력_중등중퇴',
 '아동_최종학력_초등졸업',
 '아동_최종학력_초등중퇴',
 '아동_직업유형_고등학교',
 '아동_직업유형_관리자',
 '아동_직업유형_기능원 및 관련 기능 종사자',
 '아동_직업유형_기술공 및 준전문가',
 '아동_직업유형_기타',
 '아동_직업유형_농림어업숙련종사자',
 '아동_직업유형_단순노무종사자',
 '아동_직업유형_무직',
 '아동_직업유형_사무종사자',
 '아동_직업유형_서비스 및 판매종사자',
 '아동_직업유형_어린이집',
 '아동_직업유형_유치원',
 '아동_직업유형_장치·기계조작 및 조립 종사자',
 '아동_직업유형_전문가',
 '아동_직업유형_중학교',
 '아동_직업유형_초등학교',
 '아동_거주상태_기타',
 '아동_거주상태_무상',
 '아동_거주상태_보증금(전세)+월세',
 '아동_거주상태_보호시설',
 '아동_거주상태_영구임대아파트  또는 영구임대주택',
 '아동_거주상태_월세',
 '아동_거주상태_자택',
 '아동_거주상태_전세',
 '아동_친권자유형_양모',
 '아동_친권자유형_양부',
 '아동_친권자유형_양부모',
 '아동_친권자유형_친권자 없음',
 '아동_친권자유형_친모',
 '아동_친권자유형_친부',
 '아동_친권자유형_친부모',
 '아동_친권자유형_후견인 없음',
 '아동_가족유형_기타',
 '아동_가족유형_동거(사실혼포함)',
 '아동_가족유형_모자가족(가출)',
 '아동_가족유형_모자가족(별거)',
 '아동_가족유형_모자가족(사별)',
 '아동_가족유형_모자가족(이혼)',
 '아동_가족유형_미혼부·모가정',
 '아동_가족유형_부자가족(가출)',
 '아동_가족유형_부자가족(별거)',
 '아동_가족유형_부자가족(사별)',
 '아동_가족유형_부자가족(이혼)',
 '아동_가족유형_소년소녀가정',
 '아동_가족유형_시설보호',
 '아동_가족유형_위탁가정',
 '아동_가족유형_입양가정',
 '아동_가족유형_재혼가정',
 '아동_가족유형_친부모가정',
 '아동_가족유형_친인척보호',
 '아동_다문화가족_다문화',
 '아동_다문화가족_북한이탈주민',
 '아동_다문화가족_일반',
 '아동_가구소득구분코_100만원이상-150만원미만',
 '아동_가구소득구분코_150만원이상-200만원미만',
 '아동_가구소득구분코_200만원이상-250만원미만',
 '아동_가구소득구분코_250만원이상-300만원미만',
 '아동_가구소득구분코_300만원이상',
 '아동_가구소득구분코_50만원미만',
 '아동_가구소득구분코_50만원이상~100만원미만',
 '아동_기초생활수급유_비수급권대상',
 '아동_기초생활수급유_수급권대상',
 '아동_보호조치유형_1.0',
 '신고_접수경로구분코_112',
 '신고_접수경로구분코_119',
 '신고_접수경로구분코_129',
 '신고_접수경로구분코_1366',
 '신고_접수경로구분코_기타',
 '신고_접수경로구분코_내방',
 '신고_접수경로구분코_방문신고',
 '신고_접수경로구분코_상담원 인지신고',
 '신고_접수경로구분코_아동보호전문기관일반전화',
 '신고_접수경로구분코_아동행복지원',
 '신고_접수경로구분코_아동행복지원  상담원인지신고',
 '신고_접수경로구분코_아동행복지원  읍면동인지신고',
 '신고_접수경로구분코_인터넷',
 '신고_접수경로구분코_일반전화',
 '신고_접수경로구분코_타기관통보',
 '신고_신고자유형구분_비신고의무자',
 '신고_신고자유형구분_신고의무자',
 '신고_집단시설내사건_교육기관',
 '신고_집단시설내사건_기타',
 '신고_집단시설내사건_시설',
 '신고_집단시설내사건_어린이집',
 '신고_집단시설내사건_해당사항없음',
 '신고_접수유형_아동학대 의심사례',
 '신고_접수유형_피해아동보호명령을 통한 인지',
 '신고_접수유형_학대행위자 법원명령을 통한 인지(임시조치, 보호처분)',
 '신고_신고접수구분_경찰접수',
 '신고_신고접수구분_아동보호전문기관접수',
 '신고_피해아동상태구_아동사망',
 '신고_피해아동상태구_중상해(의식불명 포함)',
 '신고_피해아동상태구_해당사항없음',
 '신대_행위자아동관계_계모',
 '신대_행위자아동관계_계부',
 '신대_행위자아동관계_기타',
 '신대_행위자아동관계_기타시설종사자',
 '신대_행위자아동관계_낯선사람',
 '신대_행위자아동관계_베이비시터(아이돌보미)',
 '신대_행위자아동관계_부모의 동거인',
 '신대_행위자아동관계_아동복지시설종사자',
 '신대_행위자아동관계_아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자',
 '신대_행위자아동관계_양모',
 '신대_행위자아동관계_양부',
 '신대_행위자아동관계_어린이집(보육교직원, 종사자)',
 '신대_행위자아동관계_외조모',
 '신대_행위자아동관계_외조부',
 '신대_행위자아동관계_위탁모',
 '신대_행위자아동관계_위탁부',
 '신대_행위자아동관계_유치원(교사,교직원, 종사자)',
 '신대_행위자아동관계_이웃',
 '신대_행위자아동관계_청소년관련시설종사자',
 '신대_행위자아동관계_친모',
 '신대_행위자아동관계_친부',
 '신대_행위자아동관계_친인척',
 '신대_행위자아동관계_친조모',
 '신대_행위자아동관계_친조부',
 '신대_행위자아동관계_학교(교원, 교직원)',
 '신대_행위자아동관계_학원(강사, 종사자)',
 '신대_행위자아동관계_형제자매',
 '신대_아동동거여부_동거',
 '신대_아동동거여부_비동거',
 '신대_접수유형_동일신고',
 '신대_접수유형_아동학대 의심사례',
 '신대_접수유형_응급아동학대의심사례',
 '조사_집단시설내사건_교육기관',
 '조사_집단시설내사건_시설',
 '조사_집단시설내사건_어린이집',
 '조사_위험_점수_1.0',
 '조사_위험_점수_2.0',
 '조사_위험_점수_3.0',
 '조사_위험_점수_4.0',
 '조사_위험_점수_5.0',
 '조사_위험_점수_6.0',
 '조사_위험_점수_7.0',
 '조사_위험_점수_8.0',
 '조사_위험_점수_9.0',
 '조대_특별관리유형_가족재결합 평가도구 사용',
 '조대_특별관리유형_공동모금회사업',
 '조대_특별관리유형_아동인권지킴이단',
 '조대_특별관리유형_종단연구',
 '판단_학대발생빈도_1개월에 한번',
 '판단_학대발생빈도_1년에 한번',
 '판단_학대발생빈도_1주일에 한번',
 '판단_학대발생빈도_2-3개월에 한번',
 '판단_학대발생빈도_2~3일에 한번',
 '판단_학대발생빈도_2년에 한번',
 '판단_학대발생빈도_2주일에 한번',
 '판단_학대발생빈도_3년에 한번',
 '판단_학대발생빈도_6개월에 한번',
 '판단_학대발생빈도_거의 매일',
 '판단_학대발생빈도_일회성',
 'New_Call_Count_1',
 'New_Call_Count_10',
 'New_Call_Count_11',
 'New_Call_Count_12',
 'New_Call_Count_13',
 'New_Call_Count_14',
 'New_Call_Count_15',
 'New_Call_Count_16',
 'New_Call_Count_17',
 'New_Call_Count_2',
 'New_Call_Count_3',
 'New_Call_Count_4',
 'New_Call_Count_5',
 'New_Call_Count_6',
 'New_Call_Count_7',
 'New_Call_Count_8',
 'New_Call_Count_9',
 '조사_조사완료여부',
 '조대_재신고여부_1',
 '조대_동일신고여부',
 '조대_아동동거여부',
 '판단_유형_신체',
 '판단_유형_정서',
 '판단_유형_성',
 '판단_유형_방임',
 '아특_공격성',
 '아특_거짓말',
 '아특_장애의심',
 '아특_배우자폭력',
 '아특_장루·요루장애',
 '아특_기타장애',
 '아특_나태(무기력)',
 '아특_언어문제',
 '아특_뇌전증장애',
 '아특_청각장애',
 '아특_늦은귀가',
 '아특_어릴적학대경험',
 '아특_기타',
 '아특_우울',
 '아특_학습문제',
 '아특_언어장애',
 '아특_성문제',
 '아특_부적절한양육태',
 '아특_호흡기장애',
 '아특_양육지식및기술',
 '아특_뇌병변장애',
 '아특_잦은결석',
 '아특_틱장애',
 '아특_인터넷(게임)',
 '아특_알콜남용',
 '아특_폭력행동',
 '아특_영양결핍',
 '아특_자폐성장애',
 '아특_위생문제',
 '아특_정신질환및장애',
 '아특_부부및가족갈등',
 '아특_대인관계기피',
 '아특_존속학대',
 '아특_주의산만',
 '아특_약물',
 '아특_대소변문제',
 '아특_안면장애',
 '아특_도박_게임중독',
 '아특_탐식및결식',
 '아특_난(難)독해',
 '아특_흡연',
 '아특_사회적고립',
 '아특_정서문제',
 '아특_지체장애',
 '아특_무력감',
 '아특_반항',
 '아특_심장장애',
 '아특_경제적어려움',
 '아특_학교부적응',
 '아특_급만성질병',
 '아특_불건전한또래관',
 '아특_잦은병치례',
 '아특_지적장애',
 '아특_신체질환및장애',
 '아특_과잉행동',
 '아특_애착문제',
 '아특_정신장애',
 '아특_신체발달지연',
 '아특_스트레스',
 '아특_낮은자아존중감',
 '아특_오락중독',
 '아특_신장장애',
 '아특_허약',
 '아특_음주',
 '아특_도벽',
 '아특_시각장애',
 '아특_불안',
 '아특_간장애',
 '아특_무단결과',
 '아특_사회성문제',
 '아특_종교문제',
 '아특_원치않는아동',
 '아특_비행집단활동',
 '아특_난(難)작문',
 '아특_충동',
 '아특_가출',
 '아특_전과력',
 '아특_특성없음',
 '아특_성격및기질문제',
 '아조_분리조치',
 '아조_원가정보호',
 '아조_가정복귀',
 '아조_사망',
 '아조_기타_1',
 '행조_고소/고발/사',
 '행조_모니터링',
 '행조_아동과의분리',
 '행조_사건처리만나지',
 '행조_만나지못함']

In [30]:
prob

,아동_성별_F,아동_성별_M,아동_생년월일_0~1세미만,아동_생년월일_10~12세,아동_생년월일_13~15세,아동_생년월일_16~17세,아동_생년월일_18~20세,아동_생년월일_1~3세,아동_생년월일_4~6세,아동_생년월일_7~9세,아동_내국인여부_내국인,아동_내국인여부_무국적,아동_내국인여부_외국인,아동_최종학력_고등졸업,아동_최종학력_고등중퇴,아동_최종학력_무학,아동_최종학력_재학중,아동_최종학력_중등졸업,아동_최종학력_중등중퇴,아동_최종학력_초등졸업,아동_최종학력_초등중퇴,아동_직업유형_고등학교,아동_직업유형_관리자,아동_직업유형_기능원 및 관련 기능 종사자,아동_직업유형_기술공 및 준전문가,아동_직업유형_기타,아동_직업유형_농림어업숙련종사자,아동_직업유형_단순노무종사자,아동_직업유형_무직,아동_직업유형_사무종사자,아동_직업유형_서비스 및 판매종사자,아동_직업유형_어린이집,아동_직업유형_유치원,아동_직업유형_장치·기계조작 및 조립 종사자,아동_직업유형_전문가,아동_직업유형_중학교,아동_직업유형_초등학교,아동_거주상태_기타,아동_거주상태_무상,아동_거주상태_보증금(전세)+월세,아동_거주상태_보호시설,아동_거주상태_영구임대아파트 또는 영구임대주택,아동_거주상태_월세,아동_거주상태_자택,아동_거주상태_전세,아동_친권자유형_양모,아동_친권자유형_양부,아동_친권자유형_양부모,아동_친권자유형_친권자 없음,아동_친권자유형_친모,아동_친권자유형_친부,아동_친권자유형_친부모,아동_친권자유형_후견인 없음,아동_가족유형_기타,아동_가족유형_동거(사실혼포함),아동_가족유형_모자가족(가출),아동_가족유형_모자가족(별거),아동_가족유형_모자가족(사별),아동_가족유형_모자가족(이혼),아동_가족유형_미혼부·모가정,아동_가족유형_부자가족(가출),아동_가족유형_부자가족(별거),아동_가족유형_부자가족(사별),아동_가족유형_부자가족(이혼),아동_가족유형_소년소녀가정,아동_가족유형_시설보호,아동_가족유형_위탁가정,아동_가족유형_입양가정,아동_가족유형_재혼가정,아동_가족유형_친부모가정,아동_가족유형_친인척보호,아동_다문화가족_다문화,아동_다문화가족_북한이탈주민,아동_다문화가족_일반,아동_가구소득구분코_100만원이상-150만원미만,아동_가구소득구분코_150만원이상-200만원미만,아동_가구소득구분코_200만원이상-250만원미만,아동_가구소득구분코_250만원이상-300만원미만,아동_가구소득구분코_300만원이상,아동_가구소득구분코_50만원미만,아동_가구소득구분코_50만원이상~100만원미만,아동_기초생활수급유_비수급권대상,아동_기초생활수급유_수급권대상,아동_보호조치유형_1.0,신고_접수경로구분코_112,신고_접수경로구분코_119,신고_접수경로구분코_129,신고_접수경로구분코_1366,신고_접수경로구분코_기타,신고_접수경로구분코_내방,신고_접수경로구분코_방문신고,신고_접수경로구분코_상담원 인지신고,신고_접수경로구분코_아동보호전문기관일반전화,신고_접수경로구분코_아동행복지원,신고_접수경로구분코_아동행복지원 상담원인지신고,신고_접수경로구분코_아동행복지원 읍면동인지신고,신고_접수경로구분코_인터넷,신고_접수경로구분코_일반전화,신고_접수경로구분코_타기관통보,신고_신고자유형구분_비신고의무자,신고_신고자유형구분_신고의무자,신고_집단시설내사건_교육기관,신고_집단시설내사건_기타,신고_집단시설내사건_시설,신고_집단시설내사건_어린이집,신고_집단시설내사건_해당사항없음,신고_접수유형_아동학대 의심사례,신고_접수유형_피해아동보호명령을 통한 인지,"신고_접수유형_학대행위자 법원명령을 통한 인지(임시조치, 보호처분)",신고_신고접수구분_경찰접수,신고_신고접수구분_아동보호전문기관접수,신고_피해아동상태구_아동사망,신고_피해아동상태구_중상해(의식불명 포함),신고_피해아동상태구_해당사항없음,신대_행위자아동관계_계모,신대_행위자아동관계_계부,신대_행위자아동관계_기타,신대_행위자아동관계_기타시설종사자,신대_행위자아동관계_낯선사람,신대_행위자아동관계_베이비시터(아이돌보미),신대_행위자아동관계_부모의 동거인,신대_행위자아동관계_아동복지시설종사자,신대_행위자아동관계_아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자,신대_행위자아동관계_양모,신대_행위자아동관계_양부,"신대_행위자아동관계_어린이집(보육교직원, 종사자)",신대_행위자아동관계_외조모,신대_행위자아동관계_외조부,신대_행위자아동관계_위탁모,신대_행위자아동관계_위탁부,"신대_행위자아동관계_유치원(교사,교직원, 종사자)",신대_행위자아동관계_이웃,신대_행위자아동관계_청소년관련시설종사자,신대_행위자아동관계_친모,신대_행위자아동관계_친부,신대_행위자아동관계_친인척,신대_행위자아동관계_친조모,신대_행위자아동관계_친조부,"신대_행위자아동관계_학교(교원, 교직원)","신대_행위자아동관계_학원(강사, 종사자)",신대_행위자아동관계_형제자매,신대_아동동거여부_동거,신대_아동동거여부_비동거,신대_접수유형_동일신고,신대_접수유형_아동학대 의심사례,신대_접수유형_응급아동학대의심사례,조사_집단시설내사건_교육기관,조사_집단시설내사건_시설,조사_집단시설내사건_어린이집,조사_위험_점수_1.0,조사_위험_점수_2.0,조사_위험_점수_3.0,조사_위험_점수_4.0,조사_위험_점수_5.0,조사_위험_점수_6.0,조사_위험_점수_7.0,조사_위험_점수_8.0,조사_위험_점수_9.0,조대_특별관리유형_가족재결합 평가도구 사용,조대_특별관리유형_공동모금회사업,조대_특별관리유형_아동인권지킴이단,조대_특별관리유형_종단연구,판단_학대발생빈도_1개월에 한번,판단_학대발생빈도_1년에 한번,판단_학대발생빈도_1주일에 한번,판단_학대발생빈도_2-3개월에 한번,판단_학대발생빈도_2~3일에 한번,판단_학대발생빈도_2년에 한번,판단_학대발생빈도_2주일에 한번,판단_학대발생빈도_3년에 한번,판단_학대발생빈도_6개월에 한번,판단_학대발생빈도_거의 매일,판단_학대발생빈도_일회성,New_Call_Count_1,New_Call_Count_10,New_Call_Count_11,New_Call_Count_12,New_Call_Count_13,New_Call_Count_14,New_Call_Count_15,New_Call_Count_16,New_Call_Count_17,New_Call_Count_2,New_Call_Count_3,New_Call_Count_4,New_Call_Count_5,New_Call_Count_6,New_Call_Count_7,New_Call_Count_8,New_Call_Count_9,조사_조사완료여부,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,판단_유형_신체,판단_유형_정서,판단_유형_성,판단_유형_방임,아특_공격성,아특_거짓말,아특_장애의심,아특_배우자폭력,아특_장루·요루장애,아특_기타장애,아특_나태(무기력),아특_언어문제,아특_뇌전증장애,아특_청각장애,아특_늦은귀가,아특_어릴적학대경험,아특_기타,아특_우울,아특_학습문제,아특_언어장애,아특_성문제,아특_부적절한양육태,아특_호흡기장애,아특_양육지식및기술,아특_뇌병변장애,아특_잦은결석,아특_틱장애,아특_인터넷(게임),아특_알콜남용,아특_폭력행동,아특_영양결핍,아특_자폐성장애,아특_위생문제,아특_정신질환및장애,아특_부부및가족갈등,아특_대인관계기피,아특_존속학대,아특_주의산만,아특_약물,아특_대소변문제,아특_안면장애,아특_도박_게임중독,아특_탐식및결식,아특_난(難)독해,아특_흡연,아특_사회적고립,아특_정서문제,아특_지체장애,아특_무력감,아특_반항,아특_심장장애,아특_경제적어려움,아특_학교부적응,아특_급만성질병,아특_불건전한또래관,아특_잦은병치례,아특_지적장애,아특_신체질환및장애,아특_과잉행동,아특_애착문제,아특_정신장애,아특_신체발달지연,아특_스트레스,아특_낮은자아존중감,아특_오락중독,아특_신장장애,아특_허약,아특_음주,아특_도벽,아특_시각장애,아특_불안,아특_간장애,아특_무단결과,아특_사회성문제,아특_종교문제,아특_원치않는아동,아특_비행집단활동,아특_난(難)작문,아특_충동,아특_가출,아특_전과력,아특_특성없음,아특_성격및기질문제,아조_분리조치,아조_원가정보호,아조_가정복귀,아조_사망,아조_기타_1,행조_고소/고발/사,행조_모니터링,행조_아동과의분리,행조_사건처리만나지,행조_만나지못함
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [92]:
mydict = {"아동_성별": ['F', 'M'],
                 "아동_생년월일": ['0~1세미만', '10~12세', '13~15세', '16~17세', '18~20세', '1~3세', '4~6세', '7~9세'],
                 "아동_내국인여부": ['내국인', '무국적', '외국인'],
                 "아동_최종학력": ['고등졸업', '고등중퇴', '무학', '재학중', '중등졸업', '중등중퇴', '초등졸업', '초등중퇴'],
                 "아동_직업유형": ['고등학교','관리자','기능원 및 관련 기능 종사자','기술공 및 준전문가','기타','농림어업숙련종사자','단순노무종사자',
                                    '무직','사무종사자','서비스 및 판매종사자','어린이집','유치원','장치·기계조작 및 조립 종사자','전문가','중학교','초등학교'], 
                 "아동_거주상태": ['기타', '무상', '보증금(전세)+월세', '보호시설', '영구임대아파트  또는 영구임대주택', '월세', '자택', '전세'], 
                 "아동_친권자유형": ['양모', '양부', '양부모', '친권자 없음', '친모', '친부', '친부모', '후견인 없음'], 
                 "아동_가족유형": ['기타','동거(사실혼포함)','모자가족(가출)','모자가족(별거)','모자가족(사별)','모자가족(이혼)','미혼부·모가정',
                                     '부자가족(가출)','부자가족(별거)','부자가족(사별)','부자가족(이혼)','소년소녀가정','시설보호','위탁가정','입양가정',
                                     '재혼가정','친부모가정','친인척보호'],
                  "아동_다문화가족": ['다문화', '북한이탈주민', '일반'],
                 "아동_가구소득구분코": ['100만원이상-150만원미만','150만원이상-200만원미만','200만원이상-250만원미만','250만원이상-300만원미만',
                                         '300만원이상','50만원미만','50만원이상~100만원미만'], 
                 "아동_기초생활수급유": ['비수급권대상', '수급권대상'], 
                 "아동_보호조치유형": ['1.0'], 
                 "신고_접수경로구분코": ['112','119','129','1366','기타','내방','방문신고','상담원 인지신고',
                                         '아동보호전문기관일반전화','아동행복지원','아동행복지원  상담원인지신고','아동행복지원  읍면동인지신고',
                                         '인터넷','일반전화','타기관통보'], 
                 "신고_신고자유형구분": ['비신고의무자', '신고의무자'], 
                 "신고_집단시설내사건_": ['교육기관', '기타', '시설', '어린이집', '해당사항없음'], 
                 "신고_접수유형": ['아동학대 의심사례', '피해아동보호명령을 통한 인지', '학대행위자 법원명령을 통한 인지(임시조치, 보호처분)'], 
                 "신고_신고접수구분": ['경찰접수', '아동보호전문기관접수'], 
                 "신고_피해아동상태구": ['아동사망', '중상해(의식불명 포함)', '해당사항없음'], 
                 "신대_행위자아동관계": ['계모','계부','기타','기타시설종사자','낯선사람','베이비시터(아이돌보미)','부모의 동거인','아동복지시설종사자',
                                         '아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자','양모','양부','어린이집(보육교직원, 종사자)','외조모',
                                         '외조부','위탁모','위탁부','유치원(교사,교직원, 종사자)','이웃','청소년관련시설종사자','친모',
                                         '친부','친인척','친조모','친조부','학교(교원, 교직원)','학원(강사, 종사자)','형제자매'], 
                 "신대_아동동거여부": ['동거', '비동거'], 
                 "신대_접수유형": ['동일신고', '아동학대 의심사례', '응급아동학대의심사례'], 
                 "조사_집단시설내사건": ['교육기관', '시설', '어린이집'], 
                 "조사_위험": ['점수_1.0','점수_2.0','점수_3.0','점수_4.0','점수_5.0','점수_6.0',
                                 '점수_7.0','점수_8.0','점수_9.0'], 
                 "조대_특별관리유형": ['가족재결합 평가도구 사용', '공동모금회사업', '아동인권지킴이단', '종단연구'], 
                 "판단_학대발생빈도": ['1개월에 한번','1년에 한번','1주일에 한번','2-3개월에 한번',
                                         '2~3일에 한번','2년에 한번','2주일에 한번','3년에 한번','6개월에 한번',
                                         '거의 매일','일회성'],
                 "New_Call_Count": ['1','10','11','12','13','14','15','16','17','2',
                                     '3','4','5','6','7','8','9'],
                 "조사_조사완료여부": [1,0],
                 "조대_재신고여부_1": [0,1],
                 "조대_동일신고여부": [0,1],
                 "조대_아동동거여부": [1,0],
                 "판단_유형": ['신체', '정서', '성', '방임']} 
dict_df = pd.DataFrame({ key:pd.Series(value) for key, value in mydict.items() }) 
dict_df

,아동_성별,아동_생년월일,아동_내국인여부,아동_최종학력,아동_직업유형,아동_거주상태,아동_친권자유형,아동_가족유형,아동_다문화가족,아동_가구소득구분코,아동_기초생활수급유,아동_보호조치유형,신고_접수경로구분코,신고_신고자유형구분,신고_집단시설내사건_,신고_접수유형,신고_신고접수구분,신고_피해아동상태구,신대_행위자아동관계,신대_아동동거여부,신대_접수유형,조사_집단시설내사건,조사_위험,조대_특별관리유형,판단_학대발생빈도,New_Call_Count,조사_조사완료여부,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,판단_유형
0,F,0~1세미만,내국인,고등졸업,고등학교,기타,양모,기타,다문화,100만원이상-150만원미만,비수급권대상,1.0,112,비신고의무자,교육기관,아동학대 의심사례,경찰접수,아동사망,계모,동거,동일신고,교육기관,점수_1.0,가족재결합 평가도구 사용,1개월에 한번,1,1.0,0.0,0.0,1.0,신체
1,M,10~12세,무국적,고등중퇴,관리자,무상,양부,동거(사실혼포함),북한이탈주민,150만원이상-200만원미만,수급권대상,NaN,119,신고의무자,기타,피해아동보호명령을 통한 인지,아동보호전문기관접수,중상해(의식불명 포함),계부,비동거,아동학대 의심사례,시설,점수_2.0,공동모금회사업,1년에 한번,10,0.0,1.0,1.0,0.0,정서
2,NaN,13~15세,외국인,무학,기능원 및 관련 기능 종사자,보증금(전세)+월세,양부모,모자가족(가출),일반,200만원이상-250만원미만,NaN,NaN,129,NaN,시설,"학대행위자 법원명령을 통한 인지(임시조치, 보호처분)",NaN,해당사항없음,기타,NaN,응급아동학대의심사례,어린이집,점수_3.0,아동인권지킴이단,1주일에 한번,11,NaN,NaN,NaN,NaN,성
3,NaN,16~17세,NaN,재학중,기술공 및 준전문가,보호시설,친권자 없음,모자가족(별거),NaN,250만원이상-300만원미만,NaN,NaN,1366,NaN,어린이집,NaN,NaN,NaN,기타시설종사자,NaN,NaN,NaN,점수_4.0,종단연구,2-3개월에 한번,12,NaN,NaN,NaN,NaN,방임
4,NaN,18~20세,NaN,중등졸업,기타,영구임대아파트 또는 영구임대주택,친모,모자가족(사별),NaN,300만원이상,NaN,NaN,기타,NaN,해당사항없음,NaN,NaN,NaN,낯선사람,NaN,NaN,NaN,점수_5.0,NaN,2~3일에 한번,13,NaN,NaN,NaN,NaN,NaN
5,NaN,1~3세,NaN,중등중퇴,농림어업숙련종사자,월세,친부,모자가족(이혼),NaN,50만원미만,NaN,NaN,내방,NaN,NaN,NaN,NaN,NaN,베이비시터(아이돌보미),NaN,NaN,NaN,점수_6.0,NaN,2년에 한번,14,NaN,NaN,NaN,NaN,NaN
6,NaN,4~6세,NaN,초등졸업,단순노무종사자,자택,친부모,미혼부·모가정,NaN,50만원이상~100만원미만,NaN,NaN,방문신고,NaN,NaN,NaN,NaN,NaN,부모의 동거인,NaN,NaN,NaN,점수_7.0,NaN,2주일에 한번,15,NaN,NaN,NaN,NaN,NaN
7,NaN,7~9세,NaN,초등중퇴,무직,전세,후견인 없음,부자가족(가출),NaN,NaN,NaN,NaN,상담원 인지신고,NaN,NaN,NaN,NaN,NaN,아동복지시설종사자,NaN,NaN,NaN,점수_8.0,NaN,3년에 한번,16,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,사무종사자,NaN,NaN,부자가족(별거),NaN,NaN,NaN,NaN,아동보호전문기관일반전화,NaN,NaN,NaN,NaN,NaN,아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자,NaN,NaN,NaN,점수_9.0,NaN,6개월에 한번,17,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,서비스 및 판매종사자,NaN,NaN,부자가족(사별),NaN,NaN,NaN,NaN,아동행복지원,NaN,NaN,NaN,NaN,NaN,양모,NaN,NaN,NaN,NaN,NaN,거의 매일,2,NaN,NaN,NaN,NaN,NaN


In [82]:
X = pd.get_dummies(dict_df)
X

,조사_조사완료여부,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,아동_성별_F,아동_성별_M,아동_생년월일_0~1세미만,아동_생년월일_10~12세,아동_생년월일_13~15세,아동_생년월일_16~17세,아동_생년월일_18~20세,아동_생년월일_1~3세,아동_생년월일_4~6세,아동_생년월일_7~9세,아동_내국인여부_내국인,아동_내국인여부_무국적,아동_내국인여부_외국인,아동_최종학력_고등졸업,아동_최종학력_고등중퇴,아동_최종학력_무학,아동_최종학력_재학중,아동_최종학력_중등졸업,아동_최종학력_중등중퇴,아동_최종학력_초등졸업,아동_최종학력_초등중퇴,아동_직업유형_고등학교,아동_직업유형_관리자,아동_직업유형_기능원 및 관련 기능 종사자,아동_직업유형_기술공 및 준전문가,아동_직업유형_기타,아동_직업유형_농림어업숙련종사자,아동_직업유형_단순노무종사자,아동_직업유형_무직,아동_직업유형_사무종사자,아동_직업유형_서비스 및 판매종사자,아동_직업유형_어린이집,아동_직업유형_유치원,아동_직업유형_장치·기계조작 및 조립 종사자,아동_직업유형_전문가,아동_직업유형_중학교,아동_직업유형_초등학교,아동_거주상태_기타,아동_거주상태_무상,아동_거주상태_보증금(전세)+월세,아동_거주상태_보호시설,아동_거주상태_영구임대아파트 또는 영구임대주택,아동_거주상태_월세,아동_거주상태_자택,아동_거주상태_전세,아동_친권자유형_양모,아동_친권자유형_양부,아동_친권자유형_양부모,아동_친권자유형_친권자 없음,아동_친권자유형_친모,아동_친권자유형_친부,아동_친권자유형_친부모,아동_친권자유형_후견인 없음,아동_가족유형_기타,아동_가족유형_동거(사실혼포함),아동_가족유형_모자가족(가출),아동_가족유형_모자가족(별거),아동_가족유형_모자가족(사별),아동_가족유형_모자가족(이혼),아동_가족유형_미혼부·모가정,아동_가족유형_부자가족(가출),아동_가족유형_부자가족(별거),아동_가족유형_부자가족(사별),아동_가족유형_부자가족(이혼),아동_가족유형_소년소녀가정,아동_가족유형_시설보호,아동_가족유형_위탁가정,아동_가족유형_입양가정,아동_가족유형_재혼가정,아동_가족유형_친부모가정,아동_가족유형_친인척보호,아동_다문화가족_다문화,아동_다문화가족_북한이탈주민,아동_다문화가족_일반,아동_가구소득구분코_100만원이상-150만원미만,아동_가구소득구분코_150만원이상-200만원미만,아동_가구소득구분코_200만원이상-250만원미만,아동_가구소득구분코_250만원이상-300만원미만,아동_가구소득구분코_300만원이상,아동_가구소득구분코_50만원미만,아동_가구소득구분코_50만원이상~100만원미만,아동_기초생활수급유_비수급권대상,아동_기초생활수급유_수급권대상,아동_보호조치유형_1.0,신고_접수경로구분코_112,신고_접수경로구분코_119,신고_접수경로구분코_129,신고_접수경로구분코_1366,신고_접수경로구분코_기타,신고_접수경로구분코_내방,신고_접수경로구분코_방문신고,신고_접수경로구분코_상담원 인지신고,신고_접수경로구분코_아동보호전문기관일반전화,신고_접수경로구분코_아동행복지원,신고_접수경로구분코_아동행복지원 상담원인지신고,신고_접수경로구분코_아동행복지원 읍면동인지신고,신고_접수경로구분코_인터넷,신고_접수경로구분코_일반전화,신고_접수경로구분코_타기관통보,신고_신고자유형구분_비신고의무자,신고_신고자유형구분_신고의무자,신고_집단시설내사건__교육기관,신고_집단시설내사건__기타,신고_집단시설내사건__시설,신고_집단시설내사건__어린이집,신고_집단시설내사건__해당사항없음,신고_접수유형_아동학대 의심사례,신고_접수유형_피해아동보호명령을 통한 인지,"신고_접수유형_학대행위자 법원명령을 통한 인지(임시조치, 보호처분)",신고_신고접수구분_경찰접수,신고_신고접수구분_아동보호전문기관접수,신고_피해아동상태구_아동사망,신고_피해아동상태구_중상해(의식불명 포함),신고_피해아동상태구_해당사항없음,신대_행위자아동관계_계모,신대_행위자아동관계_계부,신대_행위자아동관계_기타,신대_행위자아동관계_기타시설종사자,신대_행위자아동관계_낯선사람,신대_행위자아동관계_베이비시터(아이돌보미),신대_행위자아동관계_부모의 동거인,신대_행위자아동관계_아동복지시설종사자,신대_행위자아동관계_아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자,신대_행위자아동관계_양모,신대_행위자아동관계_양부,"신대_행위자아동관계_어린이집(보육교직원, 종사자)",신대_행위자아동관계_외조모,신대_행위자아동관계_외조부,신대_행위자아동관계_위탁모,신대_행위자아동관계_위탁부,"신대_행위자아동관계_유치원(교사,교직원, 종사자)",신대_행위자아동관계_이웃,신대_행위자아동관계_청소년관련시설종사자,신대_행위자아동관계_친모,신대_행위자아동관계_친부,신대_행위자아동관계_친인척,신대_행위자아동관계_친조모,신대_행위자아동관계_친조부,"신대_행위자아동관계_학교(교원, 교직원)","신대_행위자아동관계_학원(강사, 종사자)",신대_행위자아동관계_형제자매,신대_아동동거여부_동거,신대_아동동거여부_비동거,신대_접수유형_동일신고,신대_접수유형_아동학대 의심사례,신대_접수유형_응급아동학대의심사례,조사_집단시설내사건_교육기관,조사_집단시설내사건_시설,조사_집단시설내사건_어린이집,조사_위험_점수_1.0,조사_위험_점수_2.0,조사_위험_점수_3.0,조사_위험_점수_4.0,조사_위험_점수_5.0,조사_위험_점수_6.0,조사_위험_점수_7.0,조사_위험_점수_8.0,조사_위험_점수_9.0,조대_특별관리유형_가족재결합 평가도구 사용,조대_특별관리유형_공동모금회사업,조대_특별관리유형_아동인권지킴이단,조대_특별관리유형_종단연구,판단_학대발생빈도_1개월에 한번,판단_학대발생빈도_1년에 한번,판단_학대발생빈도_1주일에 한번,판단_학대발생빈도_2-3개월에 한번,판단_학대발생빈도_2~3일에 한번,판단_학대발생빈도_2년에 한번,판단_학대발생빈도_2주일에 한번,판단_학대발생빈도_3년에 한번,판단_학대발생빈도_6개월에 한번,판단_학대발생빈도_거의 매일,판단_학대발생빈도_일회성,New_Call_Count_1,New_Call_Count_10,New_Call_Count_11,New_Call_Count_12,New_Call_Count_13,New_Call_Count_14,New_Call_Count_15,New_Call_Count_16,New_Call_Count_17,New_Call_Count_2,New_Call_Count_3,New_Call_Count_4,New_Call_Count_5,New_Call_Count_6,New_Call_Count_7,New_Call_Count_8,New_Call_Count_9,판단_유형_방임,판단_유형_성,판단_유형_신체,판단_유형_정서
0,1.0,0.0,0.0,1.0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0.0,1.0,1.0,0.0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,

In [84]:
X.columns

Index(['조사_조사완료여부', '조대_재신고여부_1', '조대_동일신고여부', '조대_아동동거여부', '아동_성별_F',
       '아동_성별_M', '아동_생년월일_0~1세미만', '아동_생년월일_10~12세', '아동_생년월일_13~15세',
       '아동_생년월일_16~17세',
       ...
       'New_Call_Count_4', 'New_Call_Count_5', 'New_Call_Count_6',
       'New_Call_Count_7', 'New_Call_Count_8', 'New_Call_Count_9', '판단_유형_방임',
       '판단_유형_성', '판단_유형_신체', '판단_유형_정서'],
      dtype='object', length=198)

In [85]:
Child_result = Child[['아특_공격성', '아특_거짓말', '아특_장애의심',
       '아특_배우자폭력', '아특_장루·요루장애', '아특_기타장애', '아특_나태(무기력)', '아특_언어문제',
       '아특_뇌전증장애', '아특_청각장애', '아특_늦은귀가', '아특_어릴적학대경험', '아특_기타', '아특_우울',
       '아특_학습문제', '아특_언어장애', '아특_성문제', '아특_부적절한양육태', '아특_호흡기장애', '아특_양육지식및기술',
       '아특_뇌병변장애', '아특_잦은결석', '아특_틱장애', '아특_인터넷(게임)', '아특_알콜남용', '아특_폭력행동',
       '아특_영양결핍', '아특_자폐성장애', '아특_위생문제', '아특_정신질환및장애', '아특_부부및가족갈등',
       '아특_대인관계기피', '아특_존속학대', '아특_주의산만', '아특_약물', '아특_대소변문제', '아특_안면장애',
       '아특_도박_게임중독', '아특_탐식및결식', '아특_난(難)독해', '아특_흡연', '아특_사회적고립', '아특_정서문제',
       '아특_지체장애', '아특_무력감', '아특_반항', '아특_심장장애', '아특_경제적어려움', '아특_학교부적응',
       '아특_급만성질병', '아특_불건전한또래관', '아특_잦은병치례', '아특_지적장애', '아특_신체질환및장애',
       '아특_과잉행동', '아특_애착문제', '아특_정신장애', '아특_신체발달지연', '아특_스트레스', '아특_낮은자아존중감',
       '아특_오락중독', '아특_신장장애', '아특_허약', '아특_음주', '아특_도벽', '아특_시각장애', '아특_불안',
       '아특_간장애', '아특_무단결과', '아특_사회성문제', '아특_종교문제', '아특_원치않는아동', '아특_비행집단활동',
       '아특_난(難)작문', '아특_충동', '아특_가출', '아특_전과력', '아특_특성없음', '아특_성격및기질문제',
       '아조_분리조치', '아조_원가정보호', '아조_가정복귀', '아조_사망', '아조_기타_1', '행조_고소/고발/사',
       '행조_모니터링', '행조_아동과의분리', '행조_사건처리만나지', '행조_만나지못함']]

In [86]:
X = pd.concat([X,Child_result],axis = 1)
len(X.columns)

287

In [87]:
count =0
for i in X.columns:
    print(count, i)
    count+=1
    

0 조사_조사완료여부
1 조대_재신고여부_1
2 조대_동일신고여부
3 조대_아동동거여부
4 아동_성별_F
5 아동_성별_M
6 아동_생년월일_0~1세미만
7 아동_생년월일_10~12세
8 아동_생년월일_13~15세
9 아동_생년월일_16~17세
10 아동_생년월일_18~20세
11 아동_생년월일_1~3세
12 아동_생년월일_4~6세
13 아동_생년월일_7~9세
14 아동_내국인여부_내국인
15 아동_내국인여부_무국적
16 아동_내국인여부_외국인
17 아동_최종학력_고등졸업
18 아동_최종학력_고등중퇴
19 아동_최종학력_무학
20 아동_최종학력_재학중
21 아동_최종학력_중등졸업
22 아동_최종학력_중등중퇴
23 아동_최종학력_초등졸업
24 아동_최종학력_초등중퇴
25 아동_직업유형_고등학교
26 아동_직업유형_관리자
27 아동_직업유형_기능원 및 관련 기능 종사자
28 아동_직업유형_기술공 및 준전문가
29 아동_직업유형_기타
30 아동_직업유형_농림어업숙련종사자
31 아동_직업유형_단순노무종사자
32 아동_직업유형_무직
33 아동_직업유형_사무종사자
34 아동_직업유형_서비스 및 판매종사자
35 아동_직업유형_어린이집
36 아동_직업유형_유치원
37 아동_직업유형_장치·기계조작 및 조립 종사자
38 아동_직업유형_전문가
39 아동_직업유형_중학교
40 아동_직업유형_초등학교
41 아동_거주상태_기타
42 아동_거주상태_무상
43 아동_거주상태_보증금(전세)+월세
44 아동_거주상태_보호시설
45 아동_거주상태_영구임대아파트  또는 영구임대주택
46 아동_거주상태_월세
47 아동_거주상태_자택
48 아동_거주상태_전세
49 아동_친권자유형_양모
50 아동_친권자유형_양부
51 아동_친권자유형_양부모
52 아동_친권자유형_친권자 없음
53 아동_친권자유형_친모
54 아동_친권자유형_친부
55 아동_친권자유형_친부모
56 아동_친권자유형_후견인 없음
57 아동_가족유형_기타
58 아동_가족유형_동거(사실혼포함)
59 아동_가족유형_모자가족(가출)
60 아동_가족

In [88]:
x = X.iloc[:,4:194]
a = X.iloc[:,:4]
b = X.iloc[:,196]
c = X.iloc[:,197]
d = X.iloc[:,195]
e = X.iloc[:,194]
y = X.iloc[:,198:]

In [89]:
X = pd.concat([x,a,b,c,d,e,y],axis=1)
len(X.columns)

287

In [90]:
count =0
for i in X.columns:
    print(count, i)
    count+=1
    

0 아동_성별_F
1 아동_성별_M
2 아동_생년월일_0~1세미만
3 아동_생년월일_10~12세
4 아동_생년월일_13~15세
5 아동_생년월일_16~17세
6 아동_생년월일_18~20세
7 아동_생년월일_1~3세
8 아동_생년월일_4~6세
9 아동_생년월일_7~9세
10 아동_내국인여부_내국인
11 아동_내국인여부_무국적
12 아동_내국인여부_외국인
13 아동_최종학력_고등졸업
14 아동_최종학력_고등중퇴
15 아동_최종학력_무학
16 아동_최종학력_재학중
17 아동_최종학력_중등졸업
18 아동_최종학력_중등중퇴
19 아동_최종학력_초등졸업
20 아동_최종학력_초등중퇴
21 아동_직업유형_고등학교
22 아동_직업유형_관리자
23 아동_직업유형_기능원 및 관련 기능 종사자
24 아동_직업유형_기술공 및 준전문가
25 아동_직업유형_기타
26 아동_직업유형_농림어업숙련종사자
27 아동_직업유형_단순노무종사자
28 아동_직업유형_무직
29 아동_직업유형_사무종사자
30 아동_직업유형_서비스 및 판매종사자
31 아동_직업유형_어린이집
32 아동_직업유형_유치원
33 아동_직업유형_장치·기계조작 및 조립 종사자
34 아동_직업유형_전문가
35 아동_직업유형_중학교
36 아동_직업유형_초등학교
37 아동_거주상태_기타
38 아동_거주상태_무상
39 아동_거주상태_보증금(전세)+월세
40 아동_거주상태_보호시설
41 아동_거주상태_영구임대아파트  또는 영구임대주택
42 아동_거주상태_월세
43 아동_거주상태_자택
44 아동_거주상태_전세
45 아동_친권자유형_양모
46 아동_친권자유형_양부
47 아동_친권자유형_양부모
48 아동_친권자유형_친권자 없음
49 아동_친권자유형_친모
50 아동_친권자유형_친부
51 아동_친권자유형_친부모
52 아동_친권자유형_후견인 없음
53 아동_가족유형_기타
54 아동_가족유형_동거(사실혼포함)
55 아동_가족유형_모자가족(가출)
56 아동_가족유형_모자가족(별거)
57 아동_가족유형_모자가족(사별)
58 아동_가족유형_모자가족(이혼)
5

In [91]:
X

,아동_성별_F,아동_성별_M,아동_생년월일_0~1세미만,아동_생년월일_10~12세,아동_생년월일_13~15세,아동_생년월일_16~17세,아동_생년월일_18~20세,아동_생년월일_1~3세,아동_생년월일_4~6세,아동_생년월일_7~9세,아동_내국인여부_내국인,아동_내국인여부_무국적,아동_내국인여부_외국인,아동_최종학력_고등졸업,아동_최종학력_고등중퇴,아동_최종학력_무학,아동_최종학력_재학중,아동_최종학력_중등졸업,아동_최종학력_중등중퇴,아동_최종학력_초등졸업,아동_최종학력_초등중퇴,아동_직업유형_고등학교,아동_직업유형_관리자,아동_직업유형_기능원 및 관련 기능 종사자,아동_직업유형_기술공 및 준전문가,아동_직업유형_기타,아동_직업유형_농림어업숙련종사자,아동_직업유형_단순노무종사자,아동_직업유형_무직,아동_직업유형_사무종사자,아동_직업유형_서비스 및 판매종사자,아동_직업유형_어린이집,아동_직업유형_유치원,아동_직업유형_장치·기계조작 및 조립 종사자,아동_직업유형_전문가,아동_직업유형_중학교,아동_직업유형_초등학교,아동_거주상태_기타,아동_거주상태_무상,아동_거주상태_보증금(전세)+월세,아동_거주상태_보호시설,아동_거주상태_영구임대아파트 또는 영구임대주택,아동_거주상태_월세,아동_거주상태_자택,아동_거주상태_전세,아동_친권자유형_양모,아동_친권자유형_양부,아동_친권자유형_양부모,아동_친권자유형_친권자 없음,아동_친권자유형_친모,아동_친권자유형_친부,아동_친권자유형_친부모,아동_친권자유형_후견인 없음,아동_가족유형_기타,아동_가족유형_동거(사실혼포함),아동_가족유형_모자가족(가출),아동_가족유형_모자가족(별거),아동_가족유형_모자가족(사별),아동_가족유형_모자가족(이혼),아동_가족유형_미혼부·모가정,아동_가족유형_부자가족(가출),아동_가족유형_부자가족(별거),아동_가족유형_부자가족(사별),아동_가족유형_부자가족(이혼),아동_가족유형_소년소녀가정,아동_가족유형_시설보호,아동_가족유형_위탁가정,아동_가족유형_입양가정,아동_가족유형_재혼가정,아동_가족유형_친부모가정,아동_가족유형_친인척보호,아동_다문화가족_다문화,아동_다문화가족_북한이탈주민,아동_다문화가족_일반,아동_가구소득구분코_100만원이상-150만원미만,아동_가구소득구분코_150만원이상-200만원미만,아동_가구소득구분코_200만원이상-250만원미만,아동_가구소득구분코_250만원이상-300만원미만,아동_가구소득구분코_300만원이상,아동_가구소득구분코_50만원미만,아동_가구소득구분코_50만원이상~100만원미만,아동_기초생활수급유_비수급권대상,아동_기초생활수급유_수급권대상,아동_보호조치유형_1.0,신고_접수경로구분코_112,신고_접수경로구분코_119,신고_접수경로구분코_129,신고_접수경로구분코_1366,신고_접수경로구분코_기타,신고_접수경로구분코_내방,신고_접수경로구분코_방문신고,신고_접수경로구분코_상담원 인지신고,신고_접수경로구분코_아동보호전문기관일반전화,신고_접수경로구분코_아동행복지원,신고_접수경로구분코_아동행복지원 상담원인지신고,신고_접수경로구분코_아동행복지원 읍면동인지신고,신고_접수경로구분코_인터넷,신고_접수경로구분코_일반전화,신고_접수경로구분코_타기관통보,신고_신고자유형구분_비신고의무자,신고_신고자유형구분_신고의무자,신고_집단시설내사건__교육기관,신고_집단시설내사건__기타,신고_집단시설내사건__시설,신고_집단시설내사건__어린이집,신고_집단시설내사건__해당사항없음,신고_접수유형_아동학대 의심사례,신고_접수유형_피해아동보호명령을 통한 인지,"신고_접수유형_학대행위자 법원명령을 통한 인지(임시조치, 보호처분)",신고_신고접수구분_경찰접수,신고_신고접수구분_아동보호전문기관접수,신고_피해아동상태구_아동사망,신고_피해아동상태구_중상해(의식불명 포함),신고_피해아동상태구_해당사항없음,신대_행위자아동관계_계모,신대_행위자아동관계_계부,신대_행위자아동관계_기타,신대_행위자아동관계_기타시설종사자,신대_행위자아동관계_낯선사람,신대_행위자아동관계_베이비시터(아이돌보미),신대_행위자아동관계_부모의 동거인,신대_행위자아동관계_아동복지시설종사자,신대_행위자아동관계_아동을 보호ㆍ양육ㆍ교육하거나 그 의무가 있는 자,신대_행위자아동관계_양모,신대_행위자아동관계_양부,"신대_행위자아동관계_어린이집(보육교직원, 종사자)",신대_행위자아동관계_외조모,신대_행위자아동관계_외조부,신대_행위자아동관계_위탁모,신대_행위자아동관계_위탁부,"신대_행위자아동관계_유치원(교사,교직원, 종사자)",신대_행위자아동관계_이웃,신대_행위자아동관계_청소년관련시설종사자,신대_행위자아동관계_친모,신대_행위자아동관계_친부,신대_행위자아동관계_친인척,신대_행위자아동관계_친조모,신대_행위자아동관계_친조부,"신대_행위자아동관계_학교(교원, 교직원)","신대_행위자아동관계_학원(강사, 종사자)",신대_행위자아동관계_형제자매,신대_아동동거여부_동거,신대_아동동거여부_비동거,신대_접수유형_동일신고,신대_접수유형_아동학대 의심사례,신대_접수유형_응급아동학대의심사례,조사_집단시설내사건_교육기관,조사_집단시설내사건_시설,조사_집단시설내사건_어린이집,조사_위험_점수_1.0,조사_위험_점수_2.0,조사_위험_점수_3.0,조사_위험_점수_4.0,조사_위험_점수_5.0,조사_위험_점수_6.0,조사_위험_점수_7.0,조사_위험_점수_8.0,조사_위험_점수_9.0,조대_특별관리유형_가족재결합 평가도구 사용,조대_특별관리유형_공동모금회사업,조대_특별관리유형_아동인권지킴이단,조대_특별관리유형_종단연구,판단_학대발생빈도_1개월에 한번,판단_학대발생빈도_1년에 한번,판단_학대발생빈도_1주일에 한번,판단_학대발생빈도_2-3개월에 한번,판단_학대발생빈도_2~3일에 한번,판단_학대발생빈도_2년에 한번,판단_학대발생빈도_2주일에 한번,판단_학대발생빈도_3년에 한번,판단_학대발생빈도_6개월에 한번,판단_학대발생빈도_거의 매일,판단_학대발생빈도_일회성,New_Call_Count_1,New_Call_Count_10,New_Call_Count_11,New_Call_Count_12,New_Call_Count_13,New_Call_Count_14,New_Call_Count_15,New_Call_Count_16,New_Call_Count_17,New_Call_Count_2,New_Call_Count_3,New_Call_Count_4,New_Call_Count_5,New_Call_Count_6,New_Call_Count_7,New_Call_Count_8,New_Call_Count_9,조사_조사완료여부,조대_재신고여부_1,조대_동일신고여부,조대_아동동거여부,판단_유형_신체,판단_유형_정서,판단_유형_성,판단_유형_방임,아특_공격성,아특_거짓말,아특_장애의심,아특_배우자폭력,아특_장루·요루장애,아특_기타장애,아특_나태(무기력),아특_언어문제,아특_뇌전증장애,아특_청각장애,아특_늦은귀가,아특_어릴적학대경험,아특_기타,아특_우울,아특_학습문제,아특_언어장애,아특_성문제,아특_부적절한양육태,아특_호흡기장애,아특_양육지식및기술,아특_뇌병변장애,아특_잦은결석,아특_틱장애,아특_인터넷(게임),아특_알콜남용,아특_폭력행동,아특_영양결핍,아특_자폐성장애,아특_위생문제,아특_정신질환및장애,아특_부부및가족갈등,아특_대인관계기피,아특_존속학대,아특_주의산만,아특_약물,아특_대소변문제,아특_안면장애,아특_도박_게임중독,아특_탐식및결식,아특_난(難)독해,아특_흡연,아특_사회적고립,아특_정서문제,아특_지체장애,아특_무력감,아특_반항,아특_심장장애,아특_경제적어려움,아특_학교부적응,아특_급만성질병,아특_불건전한또래관,아특_잦은병치례,아특_지적장애,아특_신체질환및장애,아특_과잉행동,아특_애착문제,아특_정신장애,아특_신체발달지연,아특_스트레스,아특_낮은자아존중감,아특_오락중독,아특_신장장애,아특_허약,아특_음주,아특_도벽,아특_시각장애,아특_불안,아특_간장애,아특_무단결과,아특_사회성문제,아특_종교문제,아특_원치않는아동,아특_비행집단활동,아특_난(難)작문,아특_충동,아특_가출,아특_전과력,아특_특성없음,아특_성격및기질문제,아조_분리조치,아조_원가정보호,아조_가정복귀,아조_사망,아조_기타_1,행조_고소/고발/사,행조_모니터링,행조_아동과의분리,행조_사건처리만나지,행조_만나지못함
0,1.0,0.0,1.0,0.0,0.0,0.0,0.

In [98]:
X = X[3:].reset_index()
X = X.drop(['index'],axis=1)
len(X)

87481